# Myconn

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import calendar
from dateutil.relativedelta import relativedelta
from contextlib import contextmanager
from sqlalchemy import create_engine
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import os
from modules.db_connection import get_engine
import gc
import math
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import os
from datetime import datetime
from modules.queries import df__penju, df__tokoawal, df__transaction, df__cus
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, df__aging)
from modules.queries import (df__tglso, df__tglsa, df__disc2, df__margin, df__artsizecode, df__stock_so,
                             df__stock_sa)

In [7]:
from django.db import connection
def execute_raw_sql(query, params=None):
    with connection.cursor() as cursor:
        cursor.execute(query, params or [])
        columns = [col[0] for col in cursor.description]  # Get column names from cursor description
        result = cursor.fetchall()

        # Convert result to a list of dictionaries where column names are keys
        formatted_result = [dict(zip(columns, row)) for row in result]
    return formatted_result

# Test

In [ ]:
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__sales_thru_25_count_artikel, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb, df__artikel_pb_read, 
                             df__customers_pb_read, df__cus_user, df__sales_invoice,  df__sales_bz_disc2nd,
                             df__sales_bz, df__sales_consi_disc2nd, df__sales_online_disc2nd, df__otsdm, df__otsdo,
                             df__penju_today, df__artikel_status, df__tglso_, df__tglsa_ ) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis,
                            get_artikel_md, get_new_or_changed_rows,upsert_data, process_data_cus,
                            sls_disctok,proc_pivot_table_n, proc_add_aging)

## md size 1 all store

In [ ]:
df_cus= df__cus()
# SELECT fn_sizeid, fv_artsizecode, fn_artsizeid from gis_db1.articlesize_tb
df_artsq= df__artsizecode(col='fv_artsizecode, fv_barcode, fv_sizename,fv_configname',stat='imp')
# print(df_artsq.columns)

# df_sqq24= df__Sales_thru_24('*')
df_sq25= df__sales_thru_25('*')

# df_sq24=df_sqq24.copy()
# df_sq25=df_sqq25.copy()

# df_sq24 = pd.merge(df_sq24, df_artsq, on='fv_artsizecode', how='left')
# df_sq24 = pd.merge(df_sq24,df_cus,on='fn_whconsiid',how='left')

df_sq25 = pd.merge(df_sq25, df_artsq, on='fv_artsizecode', how='left', copy=False)
df_sq25.info(memory_usage='deep')
df_sq25 = pd.merge(df_sq25,df_cus,on='fn_whconsiid',how='left', copy=False)
df_sq25.info(memory_usage='deep')
# df_sq24['fn_artsizeid']=df_sq24['fn_artsizeid'].astype(int)
df_sq25['fn_artsizeid'] = pd.to_numeric(df_sq25['fn_artsizeid'], downcast='integer')
df_sq25.info(memory_usage='deep')

# df_sq24['pk_id']=df_sq24['fn_whconsiid'].astype(str)+"/"+df_sq24['fn_artsizeid'].astype(str)
df_sq25['pk_id']=df_sq25['fn_whconsiid'].astype(str)+"/"+df_sq25['fn_artsizeid'].astype(str)
df_sq25=df_sq25.drop(columns=['fn_cusid'])
#############
# print(df_sq25.columns)  
# df_sq24 = generate_rename_dict(2024, 2024, df_sq24)
df_sq25 = generate_rename_dict(2025, 2025, df_sq25)
# kolom_baru24 = (['pk_id','toko','code toko','artikel','size'] + 
#                 [col for col in df_sq24.columns if col not in 
#                 ['pk_id','toko','code toko','artikel', 'size','barcode','fv_artsizecode','fn_artsizeid','fn_whconsiid']])

kolom_baru25 = (['pk_id','toko','code toko','artikel','size'] + 
                [col for col in df_sq25.columns if col not in 
                ['pk_id','toko','code toko','artikel', 'size','barcode','fv_artsizecode','fn_artsizeid','fn_sizeid','fn_whconsiid']])


# df_sq24 = df_sq24[kolom_baru24]
df_sq25 = df_sq25[kolom_baru25]
del df_artsq, df_cus
# gc.collect()
# df_sq24 = df_sq24.loc[~(df_sq24['artikel'].isnull())]
df_sq25 = df_sq25.loc[~(df_sq25['artikel'].isnull())]
df = proc_pivot_table_n(df_sq25,
                values='stock akhir',
                col_index=['code toko','artikel'],
                columns=['size'],
                col_ket=['fv_configname'],col_select=['code toko','artikel','status'])
df_sq25=pd.merge(df_sq25,df[['code toko','artikel','status']],how='left',on=['code toko','artikel'])
del df
# gc.collect()
df_sq25=df_sq25.rename(columns={'fv_configname':'bot/top'})
# df_result24=df_sq24.select_dtypes(include='number').sum()
df_sq25=proc_add_aging(df_sq25)

# df_sq24.to_excel('/opt/share/For MDstock artikel all store 2024.xlsx', index=False)
total_rows = len(df_sq25)
chunks = 3
chunk_size = math.ceil(total_rows / chunks)  # otomatis bagi 3

with pd.ExcelWriter('/opt/share/For MD/stock artikel all store 2025.xlsx', engine='xlsxwriter') as writer:
    for i in range(chunks):
        start_row = i * chunk_size
        end_row = min(start_row + chunk_size, total_rows)
        sheet_df = df_sq25.iloc[start_row:end_row]
        sheet_df.to_excel(writer, index=False, sheet_name=f'data_{i+1}')
# df_sq24.to_excel('/opt/share/For MDstock artikel all store 2024.xlsx', index=False)
# df_sq25.to_excel('/opt/share/For MD/stock artikel all store 2025.xlsx', index=False)

# df_result24.to_excel('df_result24.xlsx')
# df_result25.to_excel('/opt/share/PB DB/df_result25.xlsx')

In [ ]:
df_sq25=df_sq25.rename(columns={'artikel':'fv_barcode'})

In [ ]:
df_sq25=proc_add_aging(df_sq25)

# df_sq24.to_excel('/opt/share/For MDstock artikel all store 2024.xlsx', index=False)
total_rows = len(df_sq25)
chunks = 3
chunk_size = math.ceil(total_rows / chunks)  # otomatis bagi 3

# with pd.ExcelWriter('/opt/share/For MD/stock artikel all store 2025.xlsx', engine='xlsxwriter') as writer:
#     for i in range(chunks):
#         start_row = i * chunk_size
#         end_row = min(start_row + chunk_size, total_rows)
#         sheet_df = df_sq25.iloc[start_row:end_row]
#         sheet_df.to_excel(writer, index=False, sheet_name=f'data_{i+1}')

In [ ]:
query=f'''
    sELECT  fv_toko, fv_spv,fv_barcode, a.fv_artsizecode,fv_sizename,  fn_stock, fv_name_doc,fv_stat, fd_tgltrx FROM gis_local.trxstx_pb25 a
    left join artikel_pb b on a.fv_artsizecode=b.fv_artsizecode  
    left join customers_pb c on a.fn_whconsiid=c.fn_whconsiid
    where fv_spv = 'Muhamad Ali Alfaridzy'
    and fd_tgltrx between '2025-06-01' and '2025-08-22' ;
    '''
with get_engine('imp') as engine_imp:
    df = pd.read_sql(query, engine_imp)
    df.to_excel('Exekusi Ali.xlsx')

In [ ]:
df_sq25.info()

In [ ]:
col_index=['code toko','artikel']
col_ket=['fv_configname']
columns=['size']
df=df_sq25
values='stock akhir'

In [ ]:
all_sizes_bottom = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
                        '39', '40', '42', '44', 'ALLSIZE']
all_sizes_top = ['FS', 'XS', 'SS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
all_sizes = all_sizes_bottom + all_sizes_top
df_ket = df[col_index + col_ket].drop_duplicates()
# Buat pivot table
pivot_df = pd.pivot_table(
    df,
    values=values,
    index=col_index,
    columns=columns,
    aggfunc='sum',
    fill_value=0).reset_index()

# Merge kembali kolom tambahan
pivot_table = pd.merge(pivot_df, df_ket, on=col_index, how='left')
for size in all_sizes:
    if size not in pivot_table.columns:
        pivot_table[size] = 0
    pivot_table[size] = pd.to_numeric(pivot_table[size], errors='coerce').fillna(0).astype(int)

# Atur ulang urutan: non-size columns dulu, lalu size columns
size_columns = [size for size in all_sizes if size in pivot_table.columns]
other_columns = [col for col in pivot_table.columns if col not in size_columns]
pivot_table = pivot_table[other_columns + size_columns]

# Hitung status (BROKEN / SEHAT)
top_columns = ["L", "M", "S", "XL", "XS", "XXL", "XXXL"]
bottom_columns = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
                    '35', '36', '37', '38', '39', '40', '42', '44']

pivot_table['status'] = np.where(
    (pivot_table["fv_configname"] == "TOP") & (pivot_table[top_columns].gt(0).sum(axis=1) < 3), "BROKEN",
    np.where(
        (pivot_table["fv_configname"] == "BOTTOM") & (pivot_table[bottom_columns].gt(0).sum(axis=1) < 5), "BROKEN",
        "SEHAT"
    )
)

# Hitung total jumlah qty size (sumQty)
pivot_table['sumQty'] = pivot_table[size_columns].sum(axis=1)
pivot_table=pivot_table.drop(columns=['fv_configname'])

In [ ]:
df=pd.merge(df,pivot_table[['code toko','artikel','status']],how='left',on=['code toko','artikel'])

## md size 2 all artikel

In [ ]:
df_artsq= df__artsizecode(col='fv_artsizecode, fv_barcode, fv_sizename,fv_configname',stat='imp')
df_sqq24= df__sales_thru_24('*')
df_sqq25= df__sales_thru_25('*')
df_sg= df__gdmd()

df_sq24=df_sqq24.copy()
df_sq25=df_sqq25.copy()
df_sq24=df_sq24.drop(columns=[df_sq24.columns[1]])
df_sq25=df_sq25.drop(columns=[df_sq25.columns[1]])
df_sq24['fn_artsizeid']=df_sq24['fn_artsizeid'].astype(str)
df_sq25['fn_artsizeid']=df_sq25['fn_artsizeid'].astype(str)
df_sq24 = df_sq24.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq24.columns[2:]].sum().reset_index()
df_sq25 = df_sq25.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq25.columns[2:]].sum().reset_index()

# Buat dictionary rename berdasarkan data
df_sq24 = generate_rename_dict(2024, 2024, df_sq24)
df_sq25 = generate_rename_dict(2025, 2025, df_sq25)
df=pd.merge(df_sq24,df_sq25,how='outer',on=['fn_artsizeid','fv_artsizecode'])
# df['qty retur']=df['qty retur_x'].fillna(0)+df['qty retur_y'].fillna(0)
print(df.columns)
df=df.rename(columns={
    'qty retur_x': 'retur 2024',
    'qty retur_y': 'retur 2025',})
df['stock akhir']=df['stock akhir_y']
df['total sales qty']=df['total sales qty_x'].fillna(0)+df['total sales qty_y'].fillna(0)
df['total stock']=df['total sales qty']+df['stock akhir'].fillna(0)
df=df.drop(columns=['total stock_x', 'stock akhir_x', 'stock akhir_y',
                    'total stock_y','total sales qty_x','total sales qty_y'])
df_sg['fv_artsizecode']=df_sg['fv_artsizecode'].str.upper()
df = df.groupby(['fv_artsizecode'], as_index=False).sum()
df = pd.merge(df, df_sg, on='fv_artsizecode', how='outer')
df = pd.merge(df, df_artsq, on='fv_artsizecode', how='left')
df =generate_rename_dict(2025, 2025, df)
# df
df = df.loc[~(df['artikel'].isnull())]
df2=df.fillna(0)
df2=df2[df2['artikel']!=0]
kolom_baru24 = ['artikel', 'size'] + [col for col in df2.columns if col not in ['artikel', 'size','fv_artsizecode','fn_artsizeid']]
df2 = df2[kolom_baru24]

In [ ]:
query='''
    SELECT fv_artsizecode, sum(stock) stock, fv_whtypename FROM gis_local.stock_gudang_pb where fv_whtypename in ('RETUR','PUSAT','KONSINETTO','QC') group by fv_artsizecode
    '''
with get_engine('imp') as engine_imp:
    df = pd.read_sql(query, engine_imp)

In [ ]:
(df.loc[~df['fv_artsizecode'].isnull(),'stock']).sum()

In [ ]:
df_sg['stock'].sum()

In [ ]:
df['stock gudang'].sum()

In [ ]:
df.loc[(df['artikel'].isnull()) & df['stock gudang']>0,'stock gudang']

In [ ]:
df2.info()

In [ ]:
df

In [ ]:
df = proc_pivot_table_n(df2,
                    values='stock akhir',
                    col_index=['artikel'],
                    columns=['size'],
                    col_ket=['fv_configname'])
df2=pd.merge(df2,df[['artikel','status']],how='left',on=['artikel'])

In [ ]:
df_sq25

In [ ]:
df2=df2.rename(columns={'fv_configname':'bot/top'})

In [ ]:
df2=pd.merge(df2,df[['artikel','status']],how='left',on=['artikel'])
df2=df2.rename(columns={'fv_configname':'bot/top'})

In [ ]:
# stock_files = generate_stock_files(base_path, start_year, end_year)

In [ ]:
# stock_files

In [ ]:
# stock_data_25

In [ ]:
# sales_data_25

# test sales to sql

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import calendar
from contextlib import contextmanager
from sqlalchemy import create_engine
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import os
from datetime import datetime
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis)
def margin_tb(year):
    df_trx, df_artimp, df_artred, df_atrxu, df_mrgincat, df_mrgin = fetch_data_margin(year)
    df_trx_final = process_data_margin(df_trx, df_artimp, df_artred, df_atrxu, df_mrgincat, df_mrgin)
    save_to_sql(df_trx_final, f'margin{str(year)[-2:]}_pb', 'imp')

In [ ]:
year=2025
start_time2= datetime.now().strftime('%Y-%m-%d %H:%M:%S')    
print("\n[INFO] Script sales_to_sql mulai dijalankan.")
print(f"[INFO] Timestamp: {start_time2}\n")

df1= df__penju(year)
df= df1.copy()
df= clean_and_transform_data(df)
df=insert_disc2_grouping(df)
margin_tb(year)
df=insert_margin(df,year)
sum_col=['fn_jualpersize','total_penjualan2','total_disc2']
agg_col=['fn_whconsiid', 'fv_artsizecode', 'tgltrx']
fisrt_col=[col for col in df.columns if col not in sum_col + agg_col]
col_col=fisrt_col+[col for col in df.columns if col not in fisrt_col]
df=df[col_col]
df['event']=df['event'].fillna(df['status_artikel'])
df['fn_margin']=df['fn_margin'].fillna(0)
df=rename_col_py_to_sql(df).sort_values(by='fn_idurut')
df=insert_pk(df)
df['fn_nett_amount'] = (df['fn_totalpenjualan']-df['fn_total_disc2']) * (1-df['fn_margin']/100)
df['fn_sim_amount'] = np.where(df['fc_status'] == "X", df['fn_totalpenjualan'], df['fn_nett_amount'])
df['fn_dpp'] = df['fn_nett_amount']/1.11
df['fn_dpp2'] = df['fn_sim_amount']/1.11
table_name = f"pb_penjualan_{year}"
save_to_sql(df, table_name, 'imp')

add_index_and_foreign_keys(
    'imp',
    table_name,
    index_cols=["fn_whconsiid", "fv_artsizecode"],
    foreign_keys=[
        {
            "column": "fn_whconsiid",
            "ref_table": "customers_pb",
            "ref_column": "fn_whconsiid",
            "constraint_name": f"fk_whconsiid_pb_{year}"
        },
        {
            "column": "fv_artsizecode",
            "ref_table": "artikel_tb",
            "ref_column": "fv_artsizecode",
            "constraint_name": f"fk_sales_artsize_pb_{year}"
        }
    ]
)

In [ ]:
df.info()

In [ ]:
df['fn_nett_amount'] = df['fn_totalpenjualan'] * (1-df['fn_margin']/100)

In [ ]:
df

In [ ]:
save_to_sql(df, table_name, 'imp')

add_index_and_foreign_keys(
    'imp',
    table_name,
    index_cols=["fn_whconsiid", "fv_artsizecode"],
    foreign_keys=[
        {
            "column": "fn_whconsiid",
            "ref_table": "customers_pb",
            "ref_column": "fn_whconsiid",
            "constraint_name": f"fk_whconsiid_pb_{year}"
        },
        {
            "column": "fv_artsizecode",
            "ref_table": "artikel_pb",
            "ref_column": "fv_artsizecode",
            "constraint_name": f"fk_artsize_pb_{year}"
        }
    ]
)

In [ ]:
# _df_cek = define_sql_types(df)

In [ ]:
# _df_cek

In [ ]:
df.info()

In [ ]:
from modules.queries import (df__tglso, df__tglsa, df__disc2, df__margin, df__artsizecode, df__stock_so,
                             df__stock_sa, df__retur, df__size, generate_months_for_year, df__rtt)
from modules.db_connection import get_engine
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data)
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import pandas as pd
from datetime import datetime
import os

In [ ]:
def trx_rt():
    df_trxstx= df__rtt()
    df_trxstx['tgltrx'] = pd.to_datetime(df_trxstx['tgltrx'], format='%m-%d-%Y')
    df_trxstx['fn_whconsiid']=df_trxstx['fn_whconsiid'].astype(int)
    df_trxstx['stock']=df_trxstx['stock'].astype(int)
    df_trxstx=df_trxstx.rename(columns={
        'stock':'fn_stock', 
        'tgltrx':'fd_tgltrx', 
        'stat':'fv_stat'
    })
    save_to_sql(df_trxstx, 'trxrt_pb', 'imp')

In [ ]:
trx_rt()

In [ ]:
from modules.db_connection import get_engine
from datetime import datetime, date
import pandas as pd

In [ ]:
today_str = date.today().strftime('%Y-%m-%d')
with get_engine('write') as engine:
        query=f'''
        SELECT fn_whconsiid, fv_artsizecode, IFNULL(SUM(totalplus), 0) AS stock, tgltrx, stat
    FROM (
    select
        SUBSTRING_INDEX(a.fv_noreceiving, '/', -1) AS fn_whconsiid,
        b.fv_artsizecode, 
        b.fn_qtyterima AS totalplus,
        DATE_FORMAT(a.fd_tglreceive, '%%m-%%d-%%Y') tgltrx, 'recieve' as stat
        FROM     t_receivingmst a
        INNER JOIN t_receivingdtl b ON a.fv_noreceiving = b.fv_noreceiving
        WHERE a.fc_status <> 'F'
        AND (a.fd_tglsistem BETWEEN '{today_str} 00:00:00' and '{today_str} 23:59:00'))
        AS subquery
    GROUP BY fn_whconsiid, fv_artsizecode, tgltrx;'''
        df= pd.read_sql(query, engine)

In [ ]:
from main import margin_tb

In [ ]:
margin_tb(2025)

# AS1 cosiment sales mini

In [ ]:
# from modules.proses import 
from modules.queries import df__sales_, df__customers_pb, df__artikel_pb
from modules.proses import rename_col_sql_to_xlsx_gis


In [ ]:
df=df__sales_(2023,'fn_whconsiid,fv_artsizecode,fv_event,fd_tgltrx,fn_jualpersize,fn_totalpenjualan,fn_nett_amount')
df_artpb=df__artikel_pb('fv_artsizecode,fv_barcode,fv_catname,fv_fitname,fv_brandname,fv_divname,fv_sizename,fm_price')

In [ ]:
df_cuspb=df__customers_pb('fv_toko,fc_code,fn_whconsiid')

In [ ]:
df=pd.merge(df, df_artpb, on='fv_artsizecode', how='left')
df=pd.merge(df, df_cuspb, on='fn_whconsiid', how='left')


In [ ]:
df=df[['fd_tgltrx','fc_code','fv_toko','fv_barcode','fv_catname','fv_fitname','fv_brandname',
    'fv_divname','fv_sizename','fv_event','fm_price','fn_totalpenjualan','fn_nett_amount','fn_jualpersize']]

In [ ]:
df=rename_col_sql_to_xlsx_gis(df)
# name_col = {
#         'fd_tgltrx': 'Sales Date',
#         'fc_code': 'Code Store',
#         'fv_toko': 'Store',
#         'fv_barcode':'Code Articel',
#         'fv_catname': 'Kategori',
#         'fv_fitname': 'Name Articel',
#         'fv_brandname': 'Brand',
#         'fv_divname': 'Division',
#         'fv_sizename': 'Size',
#         'fv_event': 'Status Article',
#         'fm_price': 'HET',
#         'fn_totalpenjualan': 'Total Nett Kassa',
#         'fn_nett_amount': 'Nett Amount',
#         'fn_jualpersize':'Qty'
#     }
#     # Lakukan rename dan sort
# df = df.rename(columns=name_col)


In [ ]:
output_folder = r'\\192.168.1.254\Share\_From Valdi\For Alokator\Sales cosiment\2023'

In [ ]:
df['Sales Date'] = pd.to_datetime(df['Sales Date'])

In [ ]:
# Tambahkan kolom bulan (format YYYY-MM)
df['bulan_num'] = df['Sales Date'].dt.strftime('%m')
df['bulan'] = df['Sales Date'].dt.strftime('%b')  # Nama bulan 3 huruf
df['tahun'] = df['Sales Date'].dt.strftime('%y')  # Tahun dua digit

# Simpan ke Excel per bulan
for (bulan_num, bulan, tahun), group in df.groupby(['bulan_num', 'bulan', 'tahun']):
    nama_file = f"{output_folder}\Sale Consi {bulan_num} {bulan} {tahun}.xlsx"
    group.drop(columns=['bulan', 'tahun', 'bulan_num']).to_excel(nama_file, index=False)
    print(f"✅ File disimpan: {nama_file}")

# AS2 Consiment Report

## Start

In [ ]:
from modules.db_connection import get_engine
from datetime import datetime, date
import pandas as pd
import numpy as np

In [ ]:
from modules.queries import (fetch_data_margin,df__artikel_pb, df__customers_pb,df__disc2)
from modules.proses import (save_to_sql, clean_and_transform_data,insert_disc2_grouping, insert_margin, process_data_margin, rename_col_py_to_sql, insert_pk,)  

In [ ]:
# Input
bulan_str = 'FEB'
bulan_str = 'MAR'
year = 2025

In [ ]:
import datetime
import calendar

# 1. Konversi nama bulan ke angka
bulan_angka = datetime.datetime.strptime(bulan_str, '%b').month

# 2. Dapatkan awal bulan
awal_bulan = datetime.date(year, bulan_angka, 1)

# 3. Dapatkan akhir bulan
akhir_hari = calendar.monthrange(year, bulan_angka)[1]  # jumlah hari di bulan tsb
akhir_bulan = datetime.date(year, bulan_angka, akhir_hari)
query = f'''SELECT 
    b.fn_idurut,
    a.fv_nopenjualan, 
    DATE_FORMAT(a.fd_tgljual, '%%m-%%d-%%Y') tgltrx,
    DATE_FORMAT(a.fd_tglinput, '%%m-%%d-%%Y %%H:%%i:%%s') AS tglinput,
    a.fn_cusid fn_whconsiid,
    b.fv_artsizecode,
    b.fn_hargajualawal price,
    b.fn_disc disc,
    b.fn_hargasatuan ,
    b.fn_jualpersize ,
    b.fn_hargasatuan*b.fn_jualpersize as total_penjualan2,
    e.fv_nameevent 'EVENT',
    a.fn_userid,
    b.fn_discid disc_id, 
    b.fv_namemark status_artikel, b.fv_descevent "desc",
    d.fv_namedisc 'Disc Name',
    d.fv_namemark 'event',
    b.fn_tabel,
    a.fv_nobonlantai,
    f.fv_articlename,
    b.fc_bonus,
    a.fn_grpcusid, 
    a.fv_namacust
FROM 
    t_penjualanmst a 
INNER JOIN 
    t_penjualandtl_goods b ON a.fv_nopenjualan = b.fv_nopenjualan
LEFT JOIN 
    (SELECT a.fn_discid, b.fv_namemark, a.fv_namedisc FROM discount_tb a
        INNER JOIN markdown_tb b ON a.fn_markid = b.fn_markid) d ON b.fn_discid = d.fn_discid
LEFT JOIN event_tb e ON b.fn_idevent=e.fn_idevent
LEFT JOIN article_tb f on b.fv_articlecode=f.fv_articlecode
WHERE 
    a.fv_nopenjualan LIKE 'PJ%%' and b.fc_status <> 'F' 
    and a.fc_status <> 'F' 
    and a.fd_tgljual BETWEEN '{awal_bulan} 00:00:01' and '{akhir_bulan} 23:59:59'
'''
with get_engine('read') as engine:  # Menggunakan context manager dengan with
    df = pd.read_sql(query, engine)


In [ ]:
df.info()

In [ ]:
df_consi= pd.read_csv(f'CONSI SALES {bulan_str} 2025.csv')

In [ ]:
df_consi.info()

In [ ]:
df_x=df.copy()
df11=df_x.copy()

In [ ]:
df11= clean_and_transform_data(df11)

In [ ]:
df11=insert_disc2_grouping(df11,'sum')

In [ ]:
df11.loc[df11['fv_nopenjualan']=='PJ/DPS/2503/00001/1143',['tglinput','artsize_voucher','artsize_affiliate','artsize_premi','artsize_servisfee']]

In [ ]:
# df11

In [ ]:
# margin_tb(year)
df11=insert_margin(df11,year)

In [ ]:
df_artpb=df__artikel_pb('*')
df_cuspb=df__customers_pb('*')
df_markdown= query = f'''
SELECT * FROM markdown_tb
'''
with get_engine('read') as engine:  # Menggunakan context manager dengan with
    df_markdown = pd.read_sql(query, engine)
    
query = f'''
SELECT * FROM user_tb
'''
with get_engine('read') as engine:  # Menggunakan context manager dengan with
    df_user = pd.read_sql(query, engine)

## 1.0 Margin SHUU

In [ ]:
with get_engine('read') as engine:
        query='''select a.fv_nopenjualan, fn_potonganharga, fn_totalqty qty from
        (SELECT fv_nopenjualan, fn_potonganharga FROM t_penjualanmst
        where fn_potonganharga <> 0) a 
        left join (SELECT fv_nopenjualan, sum(fn_jualpersize) fn_totalqty FROM t_penjualandtl_goods
        group by fv_nopenjualan) b on a.fv_nopenjualan=b.fv_nopenjualan
        '''
        df_disc2nd= pd.read_sql(query, engine)

with get_engine('read') as engine:
        query='''select a.fv_nopenjualan, servisfee, premi, affiliate, voucher, fn_totalqty qty from
        (SELECT fv_nopenjualan, fm_asuransi servisfee, fm_ongkir premi, fm_voucher affiliate, fm_lainlain voucher FROM gis_db1.t_penjualanpotongan
        ) a 
        left join (SELECT fv_nopenjualan, sum(fn_jualpersize) fn_totalqty FROM t_penjualandtl_goods
        group by fv_nopenjualan) b on a.fv_nopenjualan=b.fv_nopenjualan
        '''
        df_disc3nd= pd.read_sql(query, engine)
df_disc2nd=pd.concat([df_disc2nd, df_disc3nd])

# Jumlahkan semua kolom kecuali 'qty', dan ambil 'qty' pertama
agg_dict = {col: 'sum' for col in df_disc2nd.columns if col not in ['fv_nopenjualan', 'qty']}
agg_dict['qty'] = 'first'

# Grouping
df_disc2nd = df_disc2nd.groupby('fv_nopenjualan', as_index=False).agg(agg_dict)

df_disc2nd['artsize_disc2nd'] = np.where(df_disc2nd['fn_potonganharga'] == 0,0,(df_disc2nd['fn_potonganharga'] / df_disc2nd['qty']).round(2))
df_disc2nd['artsize_servisfee'] = np.where(df_disc2nd['servisfee'] == 0,0,(df_disc2nd['servisfee'] / df_disc2nd['qty']).round(2))
df_disc2nd['artsize_premi'] = np.where(df_disc2nd['premi'] == 0,0,(df_disc2nd['premi'] / df_disc2nd['qty']).round(2))
df_disc2nd['artsize_affiliate'] = np.where(df_disc2nd['affiliate'] == 0,0,(df_disc2nd['affiliate'] / df_disc2nd['qty']).round(2))
df_disc2nd['artsize_voucher'] = np.where(df_disc2nd['voucher'] == 0,0,(df_disc2nd['voucher'] / df_disc2nd['qty']))
df_disc2nd.replace([np.inf, -np.inf], 0, inplace=True) 
# # Mendapatkan nama kolom pertama dan terakhir
# col0 = df_disc2nd.columns[0]
# col_last = df_disc2nd.columns[-1]

In [ ]:
df_disc2nd = df_disc2nd.fillna(0)
# df_disc2nd = df_disc2nd.groupby(['fv_nopenjualan'], as_index=False).sum()

In [ ]:
pip install pandas==1.5.3 sqlalchemy==1.4.46


In [ ]:
df_disc2nd.loc[df_disc2nd['fv_nopenjualan']=='PJ/DPS/2501/00113/1128']

In [ ]:
save_to_sql(df_disc2nd, f'disc2artsize_pb', 'imp')

In [ ]:
df
df_disc2= df__disc2()
df = pd.merge(df, df_disc2, on='fv_nopenjualan', how='left')
df = df.loc[~df.duplicated()]
df[['artsize_disc2nd','artsize_servisfee','artsize_premi','artsize_affiliate','artsize_voucher']] = df[
    ['artsize_disc2nd','artsize_servisfee','artsize_premi','artsize_affiliate','artsize_voucher']].fillna(0)
df['total_disc2']=df['artsize_disc2nd']*df['fn_jualpersize']
sum_col=['fn_jualpersize','total_penjualan2','total_disc2',
            'artsize_servisfee','artsize_premi','artsize_affiliate','artsize_voucher']
agg_col=['fv_nopenjualan','fn_whconsiid', 'fv_artsizecode', 'tgltrx']
fisrt_col=[col for col in df.columns if col not in sum_col + agg_col]
print(f" {sum_col}")
print(f" {fisrt_col}")

In [ ]:
df

In [ ]:
df.loc[df['fv_nopenjualan']=='PJ/DPS/2503/00001/1143']

In [ ]:
df_margin= query = f'''
SELECT * FROM margin25_pb
'''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_margin = pd.read_sql(query, engine)

In [ ]:
from modules.queries import (fetch_data_margin)
df_trx, df_artimp, df_artred, df_atrxu, df_mrgincat, df_mrgin = fetch_data_margin(year)

In [ ]:
with get_engine('read') as engine:
    query_trx = f'''
        SELECT fn_idurut, fn_whconsiid, fd_tgljual fd_tgltrx, fv_artsizecode 
        FROM gis_db1.t_penjualandtl_goods where fd_tgljual BETWEEN '{year}-01-01' AND '{year}-12-31'
        '''
    df_trx = pd.read_sql(query_trx, engine)

In [ ]:
df_trxartcat = pd.merge(df_trx, df_artimp, on='fv_artsizecode', how='left')
df_trxartcat = pd.merge(df_trxartcat, df_artred, on='fv_barcode', how='left')
df_trxartcat = pd.merge(df_trxartcat, df_atrxu, on='fn_idurut', how='left')
df_trxartcat["fn_markid"] = df_trxartcat["fn_markid"].combine_first(df_trxartcat["markid2"])
df_trxartcat['fn_markid'] = df_trxartcat['fn_markid'].astype(int)
df_trxartcat["fd_tgltrx"] = pd.to_datetime(df_trxartcat["fd_tgltrx"])

# Merge with margin data
merged_df = pd.merge(df_trxartcat, df_mrgincat, on=["fn_whconsiid", "fn_catid", "fn_markid"], how="left")
filtered_df = merged_df[(merged_df["fd_tgltrx"] >= merged_df["fd_begindate"]) & (merged_df["fd_tgltrx"] <= merged_df["fd_enddate"])]
df_mgcatfin = filtered_df[['fn_idurut', 'fn_margin']]

merged_df2 = pd.merge(df_trxartcat, df_mrgin, on=["fn_whconsiid", "fn_markid"], how="left")
filtered_df2 = merged_df2[merged_df2["fd_tgltrx"] >= merged_df2["fd_effectivedate"]]
df_mgfin = filtered_df2.loc[filtered_df2.groupby(["fn_idurut", "fn_whconsiid", "fn_catid", "fn_markid"])["fd_effectivedate"].idxmax()]

# Merge final margin data
df_trx_final = df_trx[['fn_idurut']]
df_trx_final = pd.merge(df_trx_final, df_mgcatfin, on="fn_idurut", how="left")
df_trx_final = pd.merge(df_trx_final, df_mgfin[["fn_idurut", 'fn_margin']], on="fn_idurut", how="left", suffixes=("", "_from_mgfin"))
df_trx_final["fn_margin"] = df_trx_final["fn_margin"].combine_first(df_trx_final["fn_margin_from_mgfin"])
df_trx_final.drop(columns=["fn_margin_from_mgfin"], inplace=True, errors="ignore")
df_trx_final = df_trx_final.fillna(0)
df_trx_final = df_trx_final[~df_trx_final.duplicated()]
df_trx_final['fc_status'] = np.where(df_trx_final['fn_margin'] == 0, 'X', 'O')
df_trx_final2= pd.merge(df_trx_final, df_trxartcat[['fn_whconsiid','fn_markid','fn_idurut']], on='fn_idurut', how="left")
df_trx_final3= df_mrgin.sort_values('fd_effectivedate', ascending=False)
df_trx_final3= df_trx_final3.drop_duplicates(subset=['fn_whconsiid','fn_markid'], keep='first')
df_trx_final3 = df_trx_final3.groupby('fn_whconsiid')['fn_margin'].agg(lambda x: x.mode().iloc[0]).reset_index()
df_trx_final4= pd.merge(df_trx_final2, df_trx_final3, on=["fn_whconsiid"], how="left", suffixes=("", "_sim"))
df_trx_final4['fn_margin'] = df_trx_final4.apply(lambda row: row['fn_margin_sim'] if row['fn_margin'] == 0 else row['fn_margin'], axis=1)

In [ ]:
df_trx_final4.loc[df_trx_final4['fn_whconsiid']==186]

In [ ]:
save_to_sql(df_trx_final4, f'margin{str(year)[-2:]}_pb', 'imp')

In [ ]:
# df_consi.info()

In [ ]:
# df['fv_nopenjualan'].info()

## 1.1 Praproses

In [ ]:
df11['tglinput']=pd.to_datetime(df11['tglinput'])

In [ ]:
# Hitung selisih hari
df11['Day Input'] = (df11['tglinput'] - df11['tgltrx']).dt.days

# Beri skor berdasarkan selisih hari
def hitung_score(hari):
    if hari <= 3:
        return 'BAIK'
    elif 4 <= hari <= 10:
        return 'SEDANG'
    else:
        return 'BURUK'

df11['Score'] = df11['Day Input'].apply(hitung_score)


In [ ]:
df11['fv_nobonlantai'] = "'" + df11['fv_nobonlantai']
df11['fv_nobonlantai'] = df11['fv_nobonlantai'].str.replace(',', '', regex=False)
df11['fv_nobonlantai'] = df11['fv_nobonlantai'].str.replace("â€™", "’", regex=False)
df11['fv_nobonlantai'] = df11['fv_nobonlantai'].str.replace("â", "", regex=False)
df11['desc']=df11['desc'].str.replace(',', '', regex=False)
df11.loc[df11['fv_articlename']=='HIRT (CREW NECK) REGULAR','fv_articlename']= 'SHORT SLEEVE T-SHIRT (CREW NECK) REGULAR'
df11['fn_margin']= df11['fn_margin'].fillna(0)
df11['desc']= df11['desc'].fillna('None')


In [ ]:
df11.info()

In [ ]:
df11

In [ ]:
df11.loc[df11['fv_nopenjualan']=='PJ/DPS/2503/00532/325',['fv_nobonlantai','artsize_affiliate','artsize_premi','artsize_servisfee']]

In [ ]:
df.loc[df['fv_nopenjualan']=='PJ/DPS/2502/00094/1128',['fv_artsizecode','fc_bonus','fn_jualpersize','total_penjualan2']]

## 1.2 Main proses

In [ ]:
df12= df11.copy()

In [ ]:
df12.loc[df12['fv_nopenjualan']=='PJ/DPS/2503/00001/1143',['tglinput']]

In [ ]:
df12.info()

In [ ]:
df12=pd.merge(df12, df_cuspb[['fn_whconsiid','fc_code','fv_toko']], on='fn_whconsiid', how='left')
df12 = pd.merge(df12, df_artpb[['fv_artsizecode', 'fv_barcode', 'fv_catname', 'fv_fitname', 'fv_brandname', 'fv_divname', 'fv_sizename','fm_price']], on='fv_artsizecode', how='left')
df12 = pd.merge(df12, df_user[['fn_userid','fv_username']], on='fn_userid', how='left')

In [ ]:
df12['event']=df12['event'].fillna(df12['status_artikel'])
df12['event']=df12['event'].replace('DISC 50+20%','DISC 50%+20%')

In [ ]:
df12 = pd.merge(df12, df_markdown[['fv_namemark','fm_amount']], left_on='event',right_on='fv_namemark', how='left')

In [ ]:
df12['fm_price2'] = np.where(df12['event'].str.contains('SP|NP'), df12['price'], df12['fm_price'])

In [ ]:
conditions = [
    df12['fc_bonus'].str.contains('T'),                    # Kondisi 1
    df12['event'].str.contains('SP|NP'),                     # Kondisi 2
    df12['event'].str.contains('DISC 50%\\+20%') ,                # Kondisi 3
    df12['event'].str.contains('DISC 50%'),                    # Kondisi 4
    df12['event'].str.contains('POTONGAN'),                    # Kondisi 5
    df12['event'].str.contains('DISC 70%'),                    # Kondisi 6
    df12['event'].str.contains('NORMAL')                        # Kondisi 7
]
choices_price = [
    df12['fm_amount'],           # Jika kondisi 1 terpenuhi
    df12['fm_amount'],  # Jika kondisi 2 terpenuhi
    df12['fm_price'],        # Jika kondisi 3 terpenuhi
    df12['fm_price'],        # Jika kondisi 4 terpenuhi
    df12['price'],           # Jika kondisi 5 terpenuhi
    df12['fm_price'],        # Jika kondisi 6 terpenuhi
    df12['fm_price'],        # Jika kondisi 7 terpenuhi
]
choices_disc = [
    df12['disc'],           # Jika kondisi 1 terpenuhi
    df12['disc']*0,         # Jika kondisi 2 terpenuhi
    df12['fm_price']*0.6,   # Jika kondisi 3 terpenuhi
    df12['fm_price']*0.5,   # Jika kondisi 4 terpenuhi
    df12['disc'],           # Jika kondisi 5 terpenuhi
    df12['fm_price']*0.7,   # Jika kondisi 6 terpenuhi
    df12['disc']*0,         # Jika kondisi 7 terpenuhi
]
# Else: pakai nilai default, misalnya fm_price
df12['price'] = np.select(conditions, choices_price, default=df12['fm_price'])
df12['disc'] = np.select(conditions, choices_disc, default=df12['disc'])
df12['disc'] =round(df12['disc']).fillna(0).astype(int)

In [ ]:
df12.loc[df12['fc_bonus']=='T',['event','fm_amount','price','fc_bonus']]

In [ ]:
df12.info()

In [ ]:
df12['Total Brutto'] = np.where(df12['event'].str.contains('SP|NP'), 
                                df12['fm_amount']*df12['fn_jualpersize'], 
                                df12['price']*df12['fn_jualpersize'])
df12['Total Disc'] = df12['disc']*df12['fn_jualpersize']
df12['Total Net Kassa']=df12['Total Brutto']- df12['Total Disc'] - df12['total_disc2']

In [ ]:
df12['Margin Amount (pcs)'] = (df12['Total Net Kassa']/ df12['fn_jualpersize'] * df12['fn_margin'] / 100).round(0) 
df12['Margin Amount'] = (df12['Total Net Kassa'] * df12['fn_margin'] / 100).round(0)
df12['Net Amount'] = df12['Total Net Kassa'] - df12['Margin Amount']
df12['DPP'] = (df12['Net Amount'] / 1.11).round(2)
df12['Ppn'] = (df12['Net Amount'] / 1.11 * 0.11) .round(2)
df12.loc[df12['fn_grpcusid']==404,['DPP','Ppn']]=0

In [ ]:
df12['artsize_servisfee'] = df12['artsize_servisfee'] * df12['fn_jualpersize']
df12['artsize_premi'] = df12['artsize_premi'] * df12['fn_jualpersize']
df12['artsize_affiliate'] = df12['artsize_affiliate'] * df12['fn_jualpersize']

In [ ]:
df12.loc[df12['fn_grpcusid']==404,'DPP+PPN']=df12['Total Net Kassa']-df12['artsize_voucher']
df12['DPP+PPN'] = df12['DPP+PPN'].fillna(0)
df12['DPP Online'] = np.where(df12['DPP+PPN'] == 0,0,((df12['DPP+PPN'] / 1.11)).round(2).fillna(0))
df12['PPN Online'] = np.where(df12['DPP+PPN'] == 0,0,((df12['DPP+PPN'] / 1.11 * 0.11)).round(2).fillna(0))
df12.loc[df12['fn_grpcusid']==404,'Net Amount'] = (df12['DPP+PPN'] - df12['Margin Amount'] - df12['artsize_servisfee']
                                                - df12['artsize_premi'] - df12['artsize_affiliate']) 

In [ ]:
df12.loc[df12['fv_nopenjualan']=='PJ/DPS/2502/00391/1128',['event','fm_amount','DPP Online','PPN Online','DPP+PPN','Total Disc','total_disc2','Total Net Kassa']]

## 1.3 Cleaning and Set up

In [ ]:
df13=df12.copy()

In [ ]:
df13['fv_toko']=df13['fv_toko'].str.replace(',', '', regex=False)
df13.loc[df13['fn_grpcusid']==415,'fv_toko']='VALINO OUTLET'

In [ ]:
# Teks salah encoding
text = "937â€™934â€™941â€™943942â€™932â€™938â€™936"

# Perbaiki encoding: asumsikan teks dibaca sebagai latin-1, ubah ke utf-8
# fixed_text = text.encode('latin1').decode('utf-8')
clean_text = text.replace("â€™", "’")
print(clean_text)

In [ ]:
df13=df13.rename(columns={
    'fv_nopenjualan': 'No. Document',
    'tgltrx': 'Sales Date',
    # 'Day Input'
    # 'Score'
    'fc_code': 'Code Store',
    'fv_toko': 'Store',
    'fv_barcode': 'Code Article',
    'fv_artsizecode': 'Article',
    # 'fv_catname': 'Name Article',
    # 'fv_fitname': 'Name Article',
    'fv_articlename': 'Name Article',
    'fv_brandname': 'Brand',
    'fv_divname': 'Division',
    'fv_sizename': 'Size',
    'event': 'Status Article',
    'fm_price': 'HET',
    'price': 'Price',
    'disc': 'Disc',
    'fn_hargasatuan': 'Net Kassa',  
    'fn_jualpersize': 'Qty',
    'total_disc2':'Total Disc 2',
    # 'total_penjualan2': 'Total Net Kassa',  
    'fn_margin': 'Margin',
    'EVENT': 'Event',
    # 'Disc Name': 'Disc Name'
    'tglinput': 'Input Date',
    'fv_username': 'User Input',
    'desc': 'Description',
    'fn_tabel': 'Table',
    'artsize_servisfee': 'Service Fee',
    'artsize_premi': 'Premi',
    'artsize_affiliate': 'Affiliate',
    'artsize_voucher': 'Voucher',
    'fv_nobonlantai': 'No. Bon Lantai',

    
})

In [ ]:
col_df=['No. Document', 'Sales Date','Day Input','Score', 'Code Store', 'Store',
        'Code Article', 'Article', 'Name Article', 'Brand', 'Division', 'Size','Status Article',
        'HET', 'Price', 'Disc', 'Net Kassa', 'Qty','Total Brutto','Total Disc','Total Disc 2',
        'Total Net Kassa', 'Margin', 'DPP+PPN', 'DPP Online', 'PPN Online',
        'Margin Amount (pcs)','Margin Amount',
        'Service Fee','Premi','Affiliate','Voucher',
        'Net Amount','DPP','Ppn',
        'Event','Disc Name', 'Input Date', 'User Input', 'Description', 'Table', 'No. Bon Lantai',]
df13=df13[col_df]

In [ ]:
df13.info()

In [ ]:
df_consi2=df_consi.copy()

In [ ]:
df_consi.info()

In [ ]:
df_consi2=df_consi.copy()
# Misal: indeks kolom yang ingin dihapus
index_to_drop = [0,27,43,39]

# Ambil nama kolom berdasarkan index
cols_to_drop = df_consi2.columns[index_to_drop]

# Drop kolom tersebut
df_consi2 = df_consi2.drop(columns=cols_to_drop)


## 1.4 CHECKING

In [ ]:
df1=df13.sort_values(['No. Document','Article'])
df2=df_consi2
df2['Article']=df2['Article'].str.upper()
df2=df2.sort_values(['No. Document','Article'])
df2=df2.rename(columns={'Margin Amount / pcs':'Margin Amount (pcs)'})

In [ ]:
df1['Input Date'] = df1['Input Date'].dt.strftime('%#m/%#d/%Y %H:%M')
df1['Input Date']=pd.to_datetime(df1['Input Date'], format='mixed', errors='coerce')

In [ ]:
df2['Input Date']=pd.to_datetime(df2['Input Date'], format='mixed', errors='coerce')
# df2.loc[df2['No. Document']=='PJ/DPS/2503/00001/1143','Input Date']

In [ ]:
# Pastikan index dan kolomnya sama
df1 = df1.reset_index(drop=True).fillna('None')

df2 = df2.reset_index(drop=True).fillna('None')
# df1['Sales Date'] = pd.to_datetime(df1['Sales Date']).dt.strftime('%-m/%-d/%Y')
# df1['Input Date'] = pd.to_datetime(df1['Sales Date']).dt.strftime('%-m/%-d/%Y')

if not df1.columns.equals(df2.columns):
    print("⚠️ Kolom tidak sama")
    print("Kolom df1:", df1.columns)
    print("Kolom df2:", df2.columns)

# Buat DataFrame mask selisih nilai
mask = df1 != df2

# Tampilkan perbedaan (baris dan kolom)
diff = []
kolom_fix=mask.columns[(~mask).all()].tolist()
for row in range(len(df1)):
    for col in df1.columns:
        if col in kolom_fix:
            continue  # lewati kolom yang sudah fix
        if mask.loc[row, col]:
            diff.append({
                "index": row,
                "kolom": col,
                "df1": df1.loc[row, col],
                "df2": df2.loc[row, col]
            })

# Convert ke DataFrame
df_diff = pd.DataFrame(diff)

In [ ]:
def hitung_selisih(row):
    try:
        # Coba konversi ke float
        num1 = float(row['df1'])
        num2 = float(row['df2'])
        selisih = abs(num1 - num2)
        status = "OK" if selisih < 0.1 else "Not OK"
    except:
        selisih = 'N/A'
        status = "OK"
    return pd.Series([selisih, status], index=["selisih", "status"])

# Tambahkan kolom 'selisih' dan 'status'
# df_diff[['selisih', 'status']] = df_diff.apply(hitung_selisih, axis=1)

In [ ]:
# df_diff.loc[df_diff['status'] == 'Not OK', ['index', 'kolom', 'df1', 'df2', 'selisih']].sort_values(by='index')

In [ ]:
df1.info()

In [ ]:
# pd.merge(df_diff,df2[['nomor','No. Document','Article']],how='left',left_on='index',right_on='nomor').to_excel('File Diff.xlsx')

In [ ]:
df_diff

In [ ]:
df_diff.loc[df_diff['kolom']=='Net Kassa']

In [ ]:
df_diff['kolom'].value_counts()

kolom consi feb
Name Article           532
DPP Online             245 
Net Amount             169
Margin Amount (pcs)    165
Margin Amount          164
Margin                  78
DPP                     78
Ppn                     78
Total Net Kassa          8
DPP+PPN                  7
Price                    4
Size                     3

In [ ]:
df1['nomor'] = range(len(df1))
df2['nomor'] = range(len(df2))
# Fungsi untuk mengubah ke float jika bisa, kalau tidak biarkan string
def parse_number(val):
    try:
        # Cek apakah string hanya angka atau notasi ilmiah
        return float(val)
    except:
        return val  # Biarkan tetap string

# Terapkan ke kolom
df2['Article'] = df2['Article'].apply(parse_number)

In [ ]:
# Hapus spasi, newline, dan tab tersembunyi
df1.columns = df1.columns.str.replace('\n', '', regex=False).str.replace('\t', '', regex=False).str.strip()
df2.columns = df2.columns.str.replace('\n', '', regex=False).str.replace('\t', '', regex=False).str.strip()

col_merge=['nomor','No. Document','Article','Status Article'
        #    '
        #    'Qty','HET','Price',
        ]

con_on=['Total Net Kassa','Total Brutto','Total Disc','Total Disc 2']
col_cek = [fr'Premi']
col_cek_x= [col + '_x' for col in col_cek]
col_cek_y= [col + '_y' for col in col_cek]
col_all =col_merge+col_cek+con_on
df_check=pd.merge(df1[col_all],df2[col_all],how='inner',on=col_merge)


In [ ]:
# df_check=pd.merge(df1[col_all],df2[col_all],how='inner',on=col_merge)

# Ambil daftar index unik dari df_diff
# index_list = df_diff.loc[df_diff['status'] == 'Not OK','index'].unique()

# Filter df1 berdasarkan index yang ada di index_list
# df1_filtered = df1.loc[df1.index.isin(index_list)]
# df2_filtered = df2.loc[df2.index.isin(index_list)]
# df_checkall=pd.merge(df1_filtered, df2_filtered, how='outer', on=col_merge)

In [ ]:
mask = (df_check[col_cek_x].values != df_check[col_cek_y].values).any(axis=1)
df_beda=df_check[mask]

In [ ]:
# df1['Input Date'].to_excel('sec.xlsx')

In [ ]:
df_beda

In [ ]:
index_list_beda= df_beda['nomor'].unique()

In [ ]:
def hitung_selisih(row):
    try:
        # Coba konversi ke float
        num1 = float(row[col_cek_x])
        num2 = float(row[col_cek_y])
        selisih = abs(num1 - num2)
        status = "OK" if selisih < 0.1 else "Not OK"
    except:
        selisih = 'N/A'
        status = "OK"
    return pd.Series([selisih, status], index=["selisih", "status"])

In [ ]:
# Tambahkan kolom 'selisih' dan 'status'
# df_beda['']
# df_beda[['selisih', 'status']] = df_beda.apply(hitung_selisih, axis=1)

In [ ]:
df_beda=pd.merge(df_beda,df[['fv_nopenjualan','fn_grpcusid','fn_whconsiid']],how='left',left_on='No. Document',right_on='fv_nopenjualan').drop_duplicates(subset=['nomor'])

In [ ]:
df_beda['fn_grpcusid'].value_counts()

In [ ]:
dloc=((df_beda['fn_grpcusid'] != 2),'fn_grpcusid'  )
df_beda.loc[dloc].value_counts()

In [ ]:
dloc=((df_beda['fn_grpcusid'] == 2),'fn_whconsiid'  )
df_beda.loc[dloc].value_counts()

In [ ]:
df_beda['fn_grpcusid'].value_counts()

In [ ]:
df_beda

In [ ]:
# df_beda.loc[((df_beda['status']!= 'OK') ,['fn_grpcusid'])].value_counts()

In [ ]:
# df_checkall.to_excel('consi.xlsx')

In [ ]:
# df1.loc[df1['nomor'].isin(index_list_beda)].to_excel('dif1.xlsx', index=False)

In [ ]:
# df_beda.to_excel('diff.xlsx', index=False)

In [ ]:
# df1.to_excel('consi feb.xlsx', index=False)

# AS3 max time input

In [ ]:
from modules.db_connection import get_engine, get_db
from datetime import datetime, date
import pandas as pd

In [ ]:
def max_time_input(col_tgl_input,stat,table):
    db=get_db(stat)
    query = f'''
    select max({col_tgl_input}) max_input from {db}.{table}
    '''
    with get_engine(stat) as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine)
    return df

In [ ]:
max_time_input('fd_tglinput','write','t_mutasimst')

# AS4 stock ots DO/DM

In [ ]:
from modules.queries import (df__otsdo, df__otsdm, df__otsdo_rt, max_time_input, df__customers_pb, min_time_input,df__otsdo_del)
from modules.proses import (save_to_sql,add_index,add_foreign_keys,add_primary_key,upsert_data)
from modules.db_connection import get_engine, get_db
import pandas as pd

In [ ]:
with get_engine('write') as engine:
    OutstandingDO_query = '''SELECT 
                fn_whconsiid ,fv_artsizecode, sum(fn_qty) OstDO
                FROM gis_db.dodtl_tb a 
                inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
                inner join gis_db.t_ordermst c on b.fc_noreff=fv_noorder
                left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
                where  fv_noreceiving is null and fd_date <= '{input_value}'and b.fc_status="F" 
                group by fn_whconsiid, fv_artsizecode, fd_date '''
    df= pd.read_sql(OutstandingDO_query, engine)

In [ ]:
store_list=['MD563']
placeholders =  ", ".join([f"'{s}'" for s in store_list])
# df1['kode_list'] = df1['kode'].str.split(',')

# 2. Explode untuk meratakan
# df1_exploded = df1.explode('kode_list')

In [ ]:
def df__customers_pb_n(placeholders):
    query = f'''
        SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
        where fc_code in ({placeholders})
        '''
    with get_engine('imp') as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine)
    return df
df_cus=df__customers_pb_n(placeholders)

In [ ]:
df_cus

In [ ]:
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])

In [ ]:
cus_list

In [ ]:
def df__customers_pb_n(placeholders):
    query = f'''
        SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
        where fc_code in ({placeholders})
        '''
    with get_engine('imp') as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine)
    return df
df_cus=df__customers_pb_n(placeholders)
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
def change_cusid_to_whconsiid(df):
    df_cus=df__customers_pb('fn_cusid,fn_whconsiid')
    df=pd.merge(df,df_cus,how='left',on='fn_cusid')
    df['fn_cusid']=df['fn_whconsiid']
    df=df.drop(columns='fn_whconsiid')
    df=df.rename(columns={'fn_cusid':'fn_whconsiid'})
    return df

def get_ostdo_(cus_list):
    query = f'''
        SELECT fv_artsizecode,fv_barcode from artikel_pb
        '''
    with get_engine('imp') as engine:  # Menggunakan context manager dengan with
        df_art = pd.read_sql(query, engine)
    with get_engine('write') as engine:
        query= f'''SELECT 
        c.fn_cusid ,fv_artsizecode, sum(fn_qty) fn_qty
        FROM gis_db.dodtl_tb a 
        inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
        inner join gis_db.t_ordermst c on b.fc_noreff=fv_noorder
        left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
        where  fv_noreceiving is null and c.fn_cusid in ({cus_list}) and b.fc_status="F" 
        group by fn_cusid, fv_artsizecode, fd_date 
        '''
        df= pd.read_sql(query, engine)
        print(query)
    df=change_cusid_to_whconsiid(df)
    df=pd.merge(df, df_art, on='fv_artsizecode', how='left')
    df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()
    return df
# print(query)
df=get_ostdo_(cus_list)

In [ ]:
def get_ostdm_(whconsi_list):
    query = f'''
        SELECT fv_artsizecode,fv_barcode from artikel_pb
        '''
    with get_engine('imp') as engine:  # Menggunakan context manager dengan with
        df_art = pd.read_sql(query, engine)
    with get_engine('write') as engine:
        query= f'''
        SELECT 
            fn_whconsiid2 fn_whconsiid,fv_artsizecode, sum(fn_qty) fn_qty
            FROM gis_db.dodtl_tb a 
            inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
            inner join gis_db.t_mutasimst c on b.fc_noreff=c.fv_nomutasi
            left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
            where  fv_noreceiving is null and fn_whconsiid2 in ({whconsi_list})
            and b.fc_status="F" 
            group by fn_whconsiid2, fv_artsizecode, fd_date 
        '''
        df= pd.read_sql(query, engine)
    # df=change_cusid_to_whconsiid(df)
    df=pd.merge(df, df_art, on='fv_artsizecode', how='left')
    df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()
    return df
df=get_ostdm_(cus_list)

In [ ]:
query = f'''
        SELECT fv_artsizecode, fv_barcode from artikel_pb
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_art = pd.read_sql(query, engine)

In [ ]:
# OutstandingDM_df=pd.merge(OutstandingDM_df, df_art[['fv_artsizecode','fv_barcode']], on='fv_artsizecode', how='left')
# OutstandingDM_df= OutstandingDM_df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['OstDM'].sum()

In [ ]:
df=pd.merge(df, df_art[['fv_artsizecode','fv_barcode']], on='fv_artsizecode', how='left')
# df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()

In [ ]:
df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()

In [ ]:
df

In [ ]:
def change_cusid_to_whconsiid(df):
    df_cus=df__customers_pb('fn_cusid,fn_whconsiid')
    df=pd.merge(df,df_cus,how='left',on='fn_cusid')
    df['fn_cusid']=df['fn_whconsiid']
    df=df.drop(columns='fn_whconsiid')
    df=df.rename(columns={'fn_cusid':'fn_whconsiid'})
    return df

In [ ]:
max_input_df = max_time_input('fd_dateinput','imp','dt_OSDO')
max_input_str =max_input_df.iloc[0, 0]
max_date = max_input_str.strftime("%Y-%m-%d %H:%M:%S")
min_input_df = min_time_input('fd_dateinput','imp','dt_OSDO')
min_input_str =min_input_df.iloc[0, 0]
min_date = min_input_str.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
max_date

In [ ]:
df_del

In [ ]:
df_del, df_main =df__otsdo_del(max_date,min_date)

In [ ]:
df_del['pk_id']=df_del['fc_nodoc'].astype(str)+"/"+df_del['fv_artsizecode'].astype(str)

In [ ]:
df_main.loc[~df_main['pk_id'].isin(df_del['pk_id'])]

In [ ]:
df_del

In [ ]:
df=change_cusid_to_whconsiid(df)
df[['fn_qty','fn_whconsiid']]=df[['fn_qty','fn_whconsiid']].fillna(0).astype(int)
df['pk_id']=df['fc_nodoc'].astype(str)+"/"+df['fv_artsizecode'].astype(str)

In [ ]:
df

In [ ]:
# df= df__otsdo()
# df[['fn_qty','fn_whconsiid']]=df[['fn_qty','fn_whconsiid']].fillna(0).astype(int)
# df['pk_id']=df['fc_nodoc'].astype(str)+"/"+df['fv_artsizecode'].astype(str)
# engine='imp'
# table_name='dt_OSDO'
# col_pk='pk_id'
# save_to_sql(df,table_name,engine)
# add_index(engine,table_name,['fn_whconsiid','fv_artsizecode'])
# add_primary_key(engine,table_name,col_pk)

In [ ]:
max_input_df = max_time_input('fd_dateinput','imp','dt_OSDO')
max_input_str =max_input_df.iloc[0, 0]
max_date = max_input_str.strftime("%Y-%m-%d %H:%M:%S")
# max_date= '2025-01-01'
df=df__otsdo_rt(max_date)

In [ ]:
max_date

In [ ]:
df.info()

In [ ]:
df=change_cusid_to_whconsiid(df)
df[['fn_qty','fn_whconsiid']]=df[['fn_qty','fn_whconsiid']].fillna(0).astype(int)
df['pk_id']=df['fc_nodoc'].astype(str)+"/"+df['fv_artsizecode'].astype(str)
engine='imp'
table_name='dt_OSDO'
col_pk='pk_id'

In [ ]:
df.info()

In [ ]:
df

In [ ]:
upsert_data(engine,table_name,df,col_pk)

In [ ]:
df

In [ ]:
# Ini cukup:
max_input_df = max_time_input('fd_dateinput','imp','dt_OSDO')
max_date = max_input_df.iloc[0, 0]  # Sudah bertipe datetime.date

print(max_date)
print(type(max_date))


In [ ]:
from datetime import date

if isinstance(max_date, datetime):
    max_date = max_date.date()
print(type(max_input_df.iloc[0, 0]))  # Bisa datetime.date atau datetime.datetime


In [ ]:
max_date

# AS5 freestock

In [ ]:
store_list=['MD255']
placeholders =  ", ".join([f"'{s}'" for s in store_list])
def df__customers_pb_n(placeholders):
    query = f'''
        SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
        where fc_code in ({placeholders})
        '''
    # results = execute_raw_sql(query)
    # df = pd.DataFrame(results)
    with get_engine('imp') as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine) 
    return df
df_cus=df__customers_pb_n(placeholders)
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
def get_freestok_(whconsi_list):
    query = f'''
        SELECT fn_whconsiid, fn_qtykirim,fn_qtyverified , fv_artsizecode, t.fc_status from t_returdtl_pre t
        LEFT JOIN t_returmst_pre m on t.fv_noretur = m.fv_noretur
        WHERE t.fc_status in ('T','X') and fn_whconsiid in ({whconsi_list})
        '''
    # results = DatabaseConnector.execute_raw_sql(query_1, using=StockDashboardAPIView.dbsave)
    # df = pd.DataFrame(results)
    with get_engine('write') as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine) 
    df['stok_f']=np.where(df['fc_status'] == "X", df['fn_qtyverified'], df['fn_qtykirim'])  
    df=df[['fn_whconsiid','stok_f','fv_artsizecode']]
    return df
df=get_freestok_(whconsi_list)
df

In [ ]:
df=get_freestok_(whconsi_list)

In [ ]:
df

# AS5 Update Artikel

In [ ]:
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb, df__artikel_pb_read, 
                             df__customers_pb_read, df__cus_user, df__sales_invoice,  df__sales_bz_disc2nd,
                             df__sales_bz, df__sales_consi_disc2nd, df__sales_online_disc2nd, df__otsdm, df__otsdo,df__artikel_status ) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis,
                            get_artikel_md, get_new_or_changed_rows,upsert_data, process_data_cus,
                            sls_disctok,add_ornal)

In [ ]:
def get_artikel_md():
    # Load artikel data
    file_name = fr'\\192.168.1.254\Share\_From Valdi\For MD\FOLLOW UP 2023 (4) (5).xlsx'
    df_nfa = pd.read_excel(file_name, sheet_name=8, header=5).dropna(how='all')
    df_nfa= df_nfa [['Article Code','Artikel/ Detail 1']]
    df_nfa=df_nfa.loc[~df_nfa['Article Code'].isnull()]
    df_nfa= df_nfa.loc[~df_nfa['Artikel/ Detail 1'].isnull()]
    df_nfa=df_nfa.rename(columns={'Article Code':'fv_barcode','Artikel/ Detail 1':'fv_neo_link_foto'})
    df_nfa=df_nfa.loc[~df_nfa['fv_barcode'].duplicated()]   
    return df_nfa

In [ ]:
df= df__artikel_pb_read()

In [ ]:
df.info()

In [ ]:
df= df__artikel_pb_read()
df_nfa= get_artikel_md()
df=pd.merge(df,df_nfa,on='fv_barcode',how='left')
df=df.loc[~df['fv_artsizecode'].isnull()]
df2=df__artikel_pb("*")
colist=df.columns.to_list()
df[['fv_rating','fv_neo_link_foto']]=df[['fv_rating','fv_neo_link_foto']].fillna('none')
df2[['fv_rating','fv_neo_link_foto','fv_ornal']]=df2[['fv_rating','fv_neo_link_foto','fv_ornal']].fillna('none')
df['fv_ornal'] = df.apply(add_ornal, axis=1)
df3=get_new_or_changed_rows(df, df2,subset=colist)
df3=df3.fillna('none')
df_art_status=df__artikel_status()
df3=pd.merge(df3,df_art_status,on='fn_articleid',how='left')
df3=df3[['fv_barcode',
'fv_artsizecode',
'fv_catname',
'fv_brandname',
'fv_divname',
'fv_sizename',
'fm_price',
'fm_amount',
'fv_namemark',
'fv_colorname',
'fv_fitname',
'fv_tomname',
'fv_configname',
'motif',
'fv_picture',
'fv_rating',
'fm_cogs',
'fn_articleid',
'fv_neo_link_foto',
'fv_fitting',
'fv_status',
'fv_ornal']]
table_name = 'artikel_pb'

In [ ]:
df3.info()

In [ ]:
df3['fm_amount'] = pd.to_numeric(df3['fm_amount'], errors='coerce') 
df3['fm_amount'] = df3['fm_amount'].fillna(0)
df3['fm_price'] = pd.to_numeric(df3['fm_price'], errors='coerce')  # jadi NaN kalau gak valid
df3['fm_price'] = df3['fm_price'].fillna(0)


In [ ]:
df2=df__artikel_pb("*")

In [ ]:
df2.info()

In [ ]:
df2[df2.duplicated(subset=["fv_artsizecode"], keep=False)]


In [ ]:
# Cari baris yang punya spasi di awal, akhir, atau tengah
mask_spasi = df2['fv_artsizecode'].str.contains(r"\s", regex=True)

# Tampilkan artikel bermasalah
df_spasi = df2.loc[mask_spasi, ['fv_barcode','fv_artsizecode']]
print(df_spasi)

In [ ]:
df2= df__artikel_pb_read()

In [ ]:
df= df__artikel_pb("*")
# df['fv_ornal'] = df.apply(add_ornal, axis=1)

In [ ]:
df3 = (
    df.loc[
        df['fv_fitting'].notna() & df['fv_status'].notna(),
        ['fv_barcode', 'fv_fitting', 'fv_status']
    ]
    .drop_duplicates()
)


In [ ]:
# merge df1 dengan df3 di kolom 'fv_barcode'
df_merge = df.merge(df3, on='fv_barcode', how='left', suffixes=('', '_df3'))

# isi nilai null di df1 dengan dari df3
df_merge['fv_fitting'] = df_merge['fv_fitting'].combine_first(df_merge['fv_fitting_df3'])
df_merge['fv_status'] = df_merge['fv_status'].combine_first(df_merge['fv_status_df3'])

# drop kolom tambahan dari df3
df_merge = df_merge.drop(columns=['fv_fitting_df3', 'fv_status_df3'])


In [ ]:
df=df_merge

In [ ]:
df.loc[ (df['fv_barcode']=='BABC032L111')]

In [ ]:
df.info()

In [ ]:
df3['fv_ornal'] = df.apply(add_ornal, axis=1)

In [ ]:
df3.info()

In [ ]:
df.loc[df['fv_namemark']=='SP 299']

In [ ]:
df3=df[['fv_artsizecode','fv_namemark']].fillna('none')

In [ ]:
df3=df[['fv_artsizecode','fv_fitting','fv_status']].fillna('none')

In [ ]:
table_name = 'artikel_pb'
for col in df3.columns:
    if col != 'fv_artsizecode':
        df_sub = df3[['fv_artsizecode', col]].copy()
        upsert_data('imp', table_name, df_sub, primary_key='fv_artsizecode')
        print(col)

In [ ]:
upsert_data('imp', table_name, df3, primary_key="fv_artsizecode")

In [ ]:
table_name = 'artikel_pb'
def df__artikel_status():
    query='''select fn_articleid, fv_status from gis_db.articlestatus_tb'''
    with get_engine('write') as engine:  # Menggunakan context manager dengan with
        df = pd.read_sql(query, engine)
    return df

In [ ]:
df_art_status=df__artikel_status()

In [ ]:
df3=pd.merge(df3,df_art_status,on='fn_articleid',how='left')
df3=df3[['fv_barcode',
 'fv_artsizecode',
 'fv_catname',
 'fv_brandname',
 'fv_divname',
 'fv_sizename',
 'fm_price',
 'fm_amount',
 'fv_namemark',
 'fv_colorname',
 'fv_fitname',
 'fv_tomname',
 'fv_configname',
 'motif',
 'fv_picture',
 'fv_rating',
 'fm_cogs',
 'fn_articleid',
 'fv_neo_link_foto',
 'fv_fitting',
 'fv_status']]

In [ ]:
df_art_status

In [ ]:
df3.info()

# AS6 TARik stok mang

In [ ]:
from services.Queries_API import (df__customers_pb_n,df__customers_pb,df_artikel_pb_,df__main,df__aging,df__sales,
                                  df__retur,df__mutasi,df__obdo,change_cusid_to_whconsiid,df_ostdo_,df_ostdm_)
from services.Process_API import (proc_add_aging, proc_pivot_table, proc_add_ornal, proc_sales_stock,
                                   proc_add_retur, proc_add_mutasi, proc_add_obdo, proc_add_ostdo,
                                   proc_add_ostdm, proc_add_stok_future, proc_add_cus_art, proc_selected_col_detail,
                                   proc_add_summary, proc_cleaning_data_mobile_view,proc_selected_col_mobile_view)

## Input

In [ ]:
input_value = datetime.today().strftime("%Y-%m-%d")
input_date = datetime.strptime(input_value, "%Y-%m-%d")
# Konversi kembali ke string
start=datetime.today()
ends=start - relativedelta(months=3)
# start_str = start.strftime("%Y-%m-%d")
# ends_str = ends.strftime("%Y-%m-%d")
alokator=''
store_list = ['MD255']
input_param={'alokator': alokator,
'store_list': store_list,
'start': start.strftime("%Y-%m-%d"),
'end': ends.strftime("%Y-%m-%d"),
'type': 'default',
# 'format': output_format,
'input_date': input_date.strftime("%Y-%m-%d")}

In [ ]:
def df__main(input_param):
    input_value = input_param['input_date']
    store_list = input_param['store_list']
    if store_list:
        placeholders = ", ".join([f"'{s}'" for s in store_list])
    df_cus = df__customers_pb_n(placeholders)
    cus_list= df_cus['fn_cusid'].tolist()
    cus_list =  ", ".join([f"'{s}'" for s in cus_list])
    whconsi_list=df_cus['fn_whconsiid'].tolist()
    whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
    sales_thru = f"sales_thru_pb_{input_value[2:4]}"
    trxstx = f"trxstx_pb{input_value[2:4]}"
    stock = f"fn_stock_{input_value[5:7]}"

    main_query = f"""
    SELECT 
    stock_backdate.fn_whconsiid,
    SUM(stock) AS stocks,
    stock_backdate.fv_artsizecode, 
    fv_barcode, fv_sizename, fv_configname

    FROM (
    SELECT fn_whconsiid, {stock} AS stock, fv_artsizecode 
    FROM gis_local.{sales_thru}
    UNION ALL
    SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
    FROM gis_local.{trxstx} 
    WHERE fd_tgltrx BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
    UNION ALL
    SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
    FROM gis_local.trxrt_pb
    UNION ALL
    SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
    FROM gis_local.Stock_varian_25
    WHERE fd_tgl BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
    ) stock_backdate
    left join artikel_pb b on stock_backdate.fv_artsizecode = b.fv_artsizecode
    WHERE 1=1 and fn_whconsiid in ({whconsi_list})
    GROUP BY fn_whconsiid, fv_artsizecode
    """
    # df = pd.read_sql(main_query, engine_imp)
    results = DatabaseConnector.execute_raw_sql(main_query, using=dbLoad)
    df = pd.DataFrame(results)
    return df

In [ ]:
def get_detail(input_param):
    col='fv_barcode'
    type_table = 'detail'
    type = input_param['type']
    #Default API or Detail API
    if type == 'default':
        
        df_main = df__main(input_param)
        # print(f'test df_main')
        df_aging = df__aging(input_param)
        # print('test df_aging')
        df_sales = df__sales(input_param)
        # print('test df_sales')
        df_retur = df__retur(input_param)
        # print('test df_retur')
        df_mutasi = df__mutasi(input_param)
        # print('test df_mutasi')
        df_obdo = df__obdo(input_param)
        # print('test df_obdo')
        df_ostdo = df_ostdo_(input_param,col)
        # print('test df_ostdo')
        df_ostdm=df_ostdm_(input_param,col)
        # print('test df_ostdm')
        df = proc_add_aging(df_main, df_aging)
        # print('test proc_add_aging')
        pivot_table = proc_pivot_table(df)
        # print('test proc_pivot_table')
        pivot_table = proc_add_ornal(pivot_table)
        # print('test proc_add_ornal')
        pivot_table = proc_sales_stock(pivot_table, df_sales)
        # print('test proc_sales_stock')
        pivot_table = proc_add_retur(pivot_table, df_retur)
        # print('test df_osproc_add_returtdm')
        pivot_table = proc_add_mutasi(pivot_table, df_mutasi)
        # print('test proc_add_mutasi')
        pivot_table = proc_add_obdo(pivot_table, df_obdo)
        # print('test proc_add_obdo')
        pivot_table = proc_add_ostdo(pivot_table, df_ostdo,col)
        # print('test proc_add_ostdo')
        pivot_table = proc_add_ostdm(pivot_table, df_ostdm,col)
        # print('test proc_add_ostdm')
        pivot_table = proc_add_stok_future(pivot_table)
        # print('test proc_add_stok_future')
        pivot_table = proc_add_cus_art(pivot_table,input_param,type_table,col)
        # print('test proc_add_cus_art')
        pivot_table = proc_selected_col_detail(pivot_table)
        # pivot_table.info()
        return pivot_table
        

In [ ]:
df=get_detail(input_param)

# AS7 broken 2.0

In [ ]:
def get_top_sizes(row, size_columns):
    # Ambil qty per size
    size_qty = row[size_columns]
    total = size_qty.sum()
    if total == 0:
        return ""
    
    # Urutkan size dari qty terbesar
    sorted_sizes = size_qty.sort_values(ascending=False)
    
    # Ambil size sampai kumulatif >= 50%
    cum_sum = 0
    selected = []
    for size, qty in sorted_sizes.items():
        cum_sum += qty
        selected.append(size)
        if cum_sum >= total * 0.5:
            break
    
    return ", ".join(selected)


In [ ]:

from datetime import datetime, timedelta

date_now = datetime.today().strftime("%Y-%m-%d")
date_last_3m = (datetime.today() - timedelta(days=90)).strftime("%Y-%m-%d")

# periode tahun lalu
date_now_last_year = (datetime.today().replace(year=datetime.today().year-1)).strftime("%Y-%m-%d")
date_3m_last_year = ((datetime.today() + timedelta(days=90)).replace(year=datetime.today().year-1)).strftime("%Y-%m-%d")
query = f'''
        SELECT distinct fv_barcode,fv_catname
        FROM gis_local.artikel_pb
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_art = pd.read_sql(query, engine)
query = f'''
    SELECT 
    a.fv_artsizecode, 
    fn_cusid, 
    a.fn_whconsiid,
    fv_barcode,
    fv_sizename, 
    c.fv_configname,
    sum(fn_jualpersize) qty, 
    sum(fn_totalpenjualan) net_casa, 
    fd_tgltrx
    FROM gis_local.pb_penjualan_2025 a
    left join gis_local.customers_pb b on a.fn_whconsiid=b.fn_whconsiid
    left join gis_local.artikel_pb c on a.fv_artsizecode=c.fv_artsizecode
    where fd_tgltrx between '{date_last_3m}' and '{date_now}'
    group by fv_artsizecode, a.fn_whconsiid, fd_tgltrx;
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_cons25 = pd.read_sql(query, engine)
query = f'''
    SELECT 
    a.fv_artsizecode, 
    fn_cusid, 
    a.fn_whconsiid,
    fv_barcode,
    fv_sizename, 
    c.fv_configname,
    sum(fn_jualpersize) qty, 
    sum(fn_totalpenjualan) net_casa, 
    fd_tgltrx
    FROM gis_local.pb_penjualan_2024 a
    left join gis_local.customers_pb b on a.fn_whconsiid=b.fn_whconsiid
    left join gis_local.artikel_pb c on a.fv_artsizecode=c.fv_artsizecode
    where fd_tgltrx between '{date_now_last_year}' and '{date_3m_last_year}'
    group by fv_artsizecode, a.fn_whconsiid, fd_tgltrx;
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_cons24 = pd.read_sql(query, engine)

df=df_cons24
values:str='qty'
col_index:list=['fv_barcode', 'fn_whconsiid']
columns:list=['fv_sizename']
col_ket:list=['fv_configname']
    
all_sizes_bottom = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
                    '39', '40', '42', '44', 'ALLSIZE']
all_sizes_top = ['FS', 'XS', 'SS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
all_sizes = all_sizes_bottom + all_sizes_top
df_ket = df[col_index + col_ket].drop_duplicates()
# Buat pivot table
pivot_df = pd.pivot_table(
    df,
    values=values,
    index=col_index,
    columns=columns,
    aggfunc='sum',
    fill_value=0).reset_index()

# Merge kembali kolom tambahan
pivot_table = pd.merge(pivot_df, df_ket, on=col_index, how='left')
for size in all_sizes:
    if size not in pivot_table.columns:
        pivot_table[size] = 0
    pivot_table[size] = pd.to_numeric(pivot_table[size], errors='coerce').fillna(0).astype(int)

# Atur ulang urutan: non-size columns dulu, lalu size columns
size_columns = [size for size in all_sizes if size in pivot_table.columns]
other_columns = [col for col in pivot_table.columns if col not in size_columns]
pivot_table = pivot_table[other_columns + size_columns]

# Hitung status (BROKEN / SEHAT)
top_columns = ["L", "M", "S", "XL", "XS", "XXL", "XXXL"]
bottom_columns = ['27', '28', '29', '30', '31', '32', '33', '34',
                    '35', '36', '37', '38', '39', '40']

pivot_table['status'] = np.where(
    (pivot_table["fv_configname"] == "TOP") & (pivot_table[top_columns].gt(0).sum(axis=1) < 3), "BROKEN",
    np.where(
        (pivot_table["fv_configname"] == "BOTTOM") & (pivot_table[bottom_columns].gt(0).sum(axis=1) < 5), "BROKEN",
        "SEHAT"
    )
)

# Hitung total jumlah qty size (sumQty)
pivot_table['sumQty'] = pivot_table[size_columns].sum(axis=1)
pivot_table=pivot_table.drop(columns=['fv_configname'])

df=pivot_table
# size kecil = kolom 25–31 + kolom S + M
df["size_kecil"] = df['25'] + df['26'] + df["27"] + df["28"] + df['29'] + df['30'] + df["S"] + df["M"]

# size sedang = kolom 30–34 + kolom M + L
df["size_sedang"] = df["34"] + df["33"] + df['32'] + df['30'] + df["31"] + df["L"] + df["M"]

# size besar = kolom 34 ke atas + kolom L, XL, XXL
df["size_besar"] = df["35"] + df["36"] + df['37'] + df['38'] + df["39"] + df['40'] + df['42'] + df["44"] + df["L"] + df["XL"] + df["XXL"]
df_detail=df
df=df[['fv_barcode','fn_whconsiid','size_kecil','size_sedang','size_besar']]
df=pd.merge(df,df_art, how='left',on='fv_barcode')
import numpy as np

def get_dominant_size(row):
    k, s, b = row["size_kecil"], row["size_sedang"], row["size_besar"]

    # semua sama
    if k == s == b:
        return "ksb"
    # dua sama besar
    if k == s and k > b:
        return "ks"
    if k == b and k > s:
        return "kb"
    if s == b and s > k:
        return "sb"
    # salah satu paling besar
    if k > s and k > b:
        return "k"
    if s > k and s > b:
        return "s"
    if b > k and b > s:
        return "b"
    return None  # fallback

df["dominant_size"] = df.apply(get_dominant_size, axis=1)

In [ ]:

from datetime import datetime, timedelta

date_now = datetime.today().strftime("%Y-%m-%d")
date_last_3m = (datetime.today() - timedelta(days=90)).strftime("%Y-%m-%d")

# periode tahun lalu
date_now_last_year = (datetime.today().replace(year=datetime.today().year-1)).strftime("%Y-%m-%d")
date_3m_last_year = ((datetime.today() + timedelta(days=90)).replace(year=datetime.today().year-1)).strftime("%Y-%m-%d")
query = f'''
        SELECT distinct fv_barcode,fv_catname
        FROM gis_local.artikel_pb
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_art = pd.read_sql(query, engine)
query = f'''
    SELECT 
    a.fv_artsizecode, 
    fn_cusid, 
    a.fn_whconsiid,
    fv_barcode,
    fv_sizename, 
    c.fv_configname,
    sum(fn_jualpersize) qty, 
    sum(fn_totalpenjualan) net_casa, 
    fd_tgltrx
    FROM gis_local.pb_penjualan_2025 a
    left join gis_local.customers_pb b on a.fn_whconsiid=b.fn_whconsiid
    left join gis_local.artikel_pb c on a.fv_artsizecode=c.fv_artsizecode
    where fd_tgltrx between '{date_last_3m}' and '{date_now}'
    group by fv_artsizecode, a.fn_whconsiid, fd_tgltrx;
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_cons25 = pd.read_sql(query, engine)
query = f'''
    SELECT 
    a.fv_artsizecode, 
    fn_cusid, 
    a.fn_whconsiid,
    fv_catname,
    fv_sizename, 
    c.fv_configname,
    sum(fn_jualpersize) qty, 
    sum(fn_totalpenjualan) net_casa, 
    fd_tgltrx
    FROM gis_local.pb_penjualan_2024 a
    left join gis_local.customers_pb b on a.fn_whconsiid=b.fn_whconsiid
    left join gis_local.artikel_pb c on a.fv_artsizecode=c.fv_artsizecode
    where fd_tgltrx between '{date_now_last_year}' and '{date_3m_last_year}'
    group by fv_artsizecode, a.fn_whconsiid, fd_tgltrx;
        '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_cons24 = pd.read_sql(query, engine)

df=df_cons24
values:str='qty'
col_index:list=['fv_catname', 'fn_whconsiid']
columns:list=['fv_sizename']
col_ket:list=['fv_configname']
    
all_sizes_bottom = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
                    '39', '40', '42', '44', 'ALLSIZE']
all_sizes_top = ['FS', 'XS', 'SS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
all_sizes = all_sizes_bottom + all_sizes_top
df_ket = df[col_index + col_ket].drop_duplicates()
# Buat pivot table
pivot_df = pd.pivot_table(
    df,
    values=values,
    index=col_index,
    columns=columns,
    aggfunc='sum',
    fill_value=0).reset_index()

# Merge kembali kolom tambahan
pivot_table = pd.merge(pivot_df, df_ket, on=col_index, how='left')
for size in all_sizes:
    if size not in pivot_table.columns:
        pivot_table[size] = 0
    pivot_table[size] = pd.to_numeric(pivot_table[size], errors='coerce').fillna(0).astype(int)

# Atur ulang urutan: non-size columns dulu, lalu size columns
size_columns = [size for size in all_sizes if size in pivot_table.columns]
other_columns = [col for col in pivot_table.columns if col not in size_columns]
pivot_table = pivot_table[other_columns + size_columns]

# Hitung status (BROKEN / SEHAT)
top_columns = ["L", "M", "S", "XL", "XS", "XXL", "XXXL"]
bottom_columns = ['27', '28', '29', '30', '31', '32', '33', '34',
                    '35', '36', '37', '38', '39', '40']

pivot_table['status'] = np.where(
    (pivot_table["fv_configname"] == "TOP") & (pivot_table[top_columns].gt(0).sum(axis=1) < 3), "BROKEN",
    np.where(
        (pivot_table["fv_configname"] == "BOTTOM") & (pivot_table[bottom_columns].gt(0).sum(axis=1) < 5), "BROKEN",
        "SEHAT"
    )
)

# Hitung total jumlah qty size (sumQty)
pivot_table['sumQty'] = pivot_table[size_columns].sum(axis=1)
pivot_table=pivot_table.drop(columns=['fv_configname'])

df=pivot_table
# size kecil = kolom 25–31 + kolom S + M
df["size_kecil"] = df['25'] + df['26'] + df["27"] + df["28"] + df['29'] + df['30'] + df["S"] + df["M"]

# size sedang = kolom 30–34 + kolom M + L
df["size_sedang"] = df["34"] + df["33"] + df['32'] + df['30'] + df["31"] + df["L"] + df["M"]

# size besar = kolom 34 ke atas + kolom L, XL, XXL
df["size_besar"] = df["35"] + df["36"] + df['37'] + df['38'] + df["39"] + df['40'] + df['42'] + df["44"] + df["L"] + df["XL"] + df["XXL"]
df_detail=df
df=df[['fv_barcode','fn_whconsiid','size_kecil','size_sedang','size_besar']]
df=pd.merge(df,df_art, how='left',on='fv_barcode')
import numpy as np

def get_dominant_size(row):
    k, s, b = row["size_kecil"], row["size_sedang"], row["size_besar"]

    # semua sama
    if k == s == b:
        return "ksb"
    # dua sama besar
    if k == s and k > b:
        return "ks"
    if k == b and k > s:
        return "kb"
    if s == b and s > k:
        return "sb"
    # salah satu paling besar
    if k > s and k > b:
        return "k"
    if s > k and s > b:
        return "s"
    if b > k and b > s:
        return "b"
    return None  # fallback

df["dominant_size"] = df.apply(get_dominant_size, axis=1)

In [ ]:
# Terapkan ke pivot_table
df_detail["dominant_sizes"] = df_detail.apply(lambda row: get_top_sizes(row, size_columns), axis=1)

In [ ]:
df_detail[df_detail.columns[20:]].sort_values('sumQty',ascending=False)

In [ ]:
# Terapkan ke pivot_table
pivot_table["dominant_sizes"] = pivot_table.apply(lambda row: get_top_sizes(row, size_columns), axis=1)

In [ ]:
pivot_table.loc[pivot_table['dominant_sizes']=='XL, L']

In [ ]:
df_detail['dominant_sizes'].value_counts().head(20)

In [ ]:
df=df[['fv_barcode','fn_whconsiid','size_kecil','size_sedang','size_besar']]
df=pd.merge(df,df_art, how='left',on='fv_barcode')

In [ ]:
df_detail

In [ ]:
df

In [ ]:
pivot_table

In [ ]:
df_detail

In [ ]:
pivot_table

In [ ]:
df_detail

In [ ]:
# Terapkan ke pivot_table
pivot_table["dominant_sizes"] = pivot_table.apply(lambda row: get_top_sizes(row, size_columns), axis=1)

In [ ]:
df_cat_dom

In [ ]:
df_cat_dom = (
    df.groupby(['fn_whconsiid', 'fv_catname'], as_index=False)
      .agg({
          'size_kecil': 'sum',
          'size_sedang': 'sum',
          'size_besar': 'sum'
      })
)

In [ ]:
df_cat_dom["dominant_size"] = df_cat_dom.apply(get_dominant_size, axis=1)

In [ ]:
df_cat_dom['dominant_size'].value_counts()

In [ ]:
df_cat_dom.loc[(df_cat_dom['dominant_size']=='ksb'),'fv_catname'].value_counts()

In [ ]:
df

In [ ]:
df.loc[df['fn_whconsiid']==19]

# AS8 Update whtype

In [ ]:
from modules.queries import (df__whtype_name) 
from modules.proses import (save_to_sql)

from sqlalchemy import Integer, String, CHAR, DateTime, Text, text, Float, Numeric

In [ ]:
def define_sql_types(df):
    """Menentukan tipe data untuk SQL sesuai dengan tipe kolom."""
    sql_types = {}
    for col in df.columns:
        col_dtype = df[col].dtype

        # Jika kolom tipe float
        if col_dtype == 'float64':
            sql_types[col] = Numeric(15, 2)
        # Jika kolom diawali 'fn' dan bukan float
        elif col.startswith('fn'):
            sql_types[col] = Integer
        # Jika kolom diawali 'fv'
        elif col.startswith('fv'):
            max_length = df[col].astype(str).map(len).max()
            if max_length > 255:
                sql_types[col] = Text
            else:
                sql_types[col] = String(255)
        # Jika kolom diawali 'fc'
        elif col.startswith('fc'):
            sql_types[col] = CHAR(20)
        # Jika kolom diawali 'fd'
        elif col.startswith('fd'):
            sql_types[col] = DateTime
        # Kalau tidak masuk kriteria di atas
        else:
            sql_types[col] = String(50)
    
    return sql_types


def save_to_sql(df, table_name, engine):
    """Menyimpan data ke dalam database menggunakan SQLAlchemy."""
    with get_engine(engine) as engine:
        sql_types = define_sql_types(df)
        df.to_sql(table_name, con=engine, if_exists='replace', index=False, dtype=sql_types)

In [ ]:
df=df__whtype_name()
nama_tabel='whtype'
save_to_sql(df, nama_tabel, 'imp')

# AS9 Aging gudang

In [ ]:
query_gr = '''
    SELECT a.fc_nodoc, fd_arrivaldate, fv_nopo, fn_articleid, fn_qty FROM gis_db1.grdtl_tb a
    inner join gis_db1.grmst_tb b on a.fc_nodoc = b.fc_nodoc
    '''
with get_engine('read') as engine:  # Menggunakan context manager dengan with
    df_gr = pd.read_sql(query_gr, engine)
# Cari tanggal dengan qty tertinggi per store & product
result = df_gr.sort_values('fn_qty', ascending=False).groupby(['fv_nopo', 'fn_articleid']).first().reset_index()

# Urutkan dari tanggal terbaru
df_sorted = result.sort_values('fd_arrivaldate', ascending=False)

# Ambil baris pertama (arrival terbaru) untuk tiap articleid
df_latest = df_sorted.drop_duplicates(subset='fn_articleid', keep='first')

query_gr = '''
    SELECT * FROM gis_local.artikel_pb 
    '''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
    df_art = pd.read_sql(query_gr, engine)

unique_ids = pd.DataFrame(df_art['fn_articleid'].unique(), columns=['fn_articleid'])
df_latest = pd.merge(df_latest, unique_ids, on='fn_articleid', how='right')

df_latest=df_latest.fillna({'fn_qty':0, 'fd_arrivaldate':'2023-01-01', 'fv_nopo':'', 'fc_nodoc':''})


today = pd.Timestamp(datetime.now().date())
df_latest["fn_hari"] = (today - df_latest["fd_arrivaldate"]).dt.days
df_latest["fn_bulan"] = (today.year - df_latest["fd_arrivaldate"].dt.year) * 12 + (today.month - df_latest["fd_arrivaldate"].dt.month)

# df_latest.to_sql(f'aging_gudang_pb', engine_imp, if_exists='replace', index=False)

nama_tabel='aging_gudang_pb'
save_to_sql(df_latest, nama_tabel, 'imp')

# troubel

In [ ]:
d

In [1]:
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb, df__artikel_pb_read, 
                             df__customers_pb_read, df__cus_user, df__sales_invoice,  df__sales_bz_disc2nd,
                             df__sales_bz, df__sales_consi_disc2nd, df__sales_online_disc2nd, df__otsdm, df__otsdo,
                             df__artikel_status ) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis,
                            get_artikel_md, get_new_or_changed_rows,upsert_data, process_data_cus,
                            sls_disctok)
from modules.db_connection import get_engine, get_db
from main import md_artikel_size2
from datetime import date, timedelta

## stock monthly

In [ ]:
def get_stock_monthly(tglEND,expot_excel):
    '''Main Stock Monthly'''
    tglEND2= datetime.strptime(tglEND, "%Y-%m-%d").date()
    tglEND = datetime.strptime(tglEND, '%Y-%m-%d').strftime('%Y-%m-%d 23:59:59')
    # Dapatkan tanggal kemarin
    today = datetime.now()
    d2ago = today - timedelta(days=2)
    # Ubah ke string (format default: YYYY-MM-DD)
    # kemarin_str = kemarin.strftime("%Y-%m-%d")
    # d2ago_str = d2ago.strftime("%Y-%m-%d")
    year = int(tglEND[:4])
    month = int(tglEND[5:7])
    day = int(tglEND[8:10])
    last_day = calendar.monthrange(year, month)[1]
    if day == last_day:
        if month == 12:  
            month = 1
            year += 1
        else:
            month += 1
    bulan = calendar.month_name[month].lower()[:3]
    expot_excel = expot_excel.lower()
    if expot_excel not in ['yes', 'no']:
        raise ValueError("Parameter 'expot_excel' harus berupa 'yes' atau 'no'.")
    tglnow = datetime.today().date() # Tanggal Perhari Ini 
    df_tokoawal = df__tokoawal()
    idwhtoko_list_awal = df_tokoawal['fn_whconsiid'].tolist() # Ubah kedalam List

    # Mencari tanggal SO terakhir utnuk masing-masing toko yang masih buka per tanggal yang dipilih
    df_tglLSO = pd.concat([get_tgl(idwhtoko, tglnow, tglEND) for idwhtoko in idwhtoko_list_awal])
    df_tglLSO['fd_tglflagIndo'] = pd.to_datetime(df_tglLSO['fd_tglflagIndo'])

    if tglEND2 > tglnow:  # Jika tanggal yang dipilih lebih besar dari hari ini
        df_tglLSO.loc[df_tglLSO['fc_status']=="T",'fd_tglflagIndo'] -= pd.Timedelta(days=1)

    df_tglLSO['fd_tglflagIndo'] = df_tglLSO['fd_tglflagIndo'].dt.strftime('%Y-%m-%d')
    idwhtoko_list = df_tglLSO['fn_whconsiid'].tolist() # Ubah kedalam List untuk kode toko
    tgl_list = df_tglLSO['fd_tglflagIndo'].tolist() # Ubah kedalam List untuk tanggal SO terakhir

    # Mencari Stock pertanggal yang dipilih dengan menjumlah stock awal SO terakhir dengan transaksi dari SO terakhir sampai tanggal yang dipilih
    df_tok = pd.DataFrame() # Dataframe Kosong
    for idwhtoko, tglSTR in zip(idwhtoko_list, tgl_list): # Looping untuk masing-masing toko dan tanggal SO terakhir
        df_stock = get_stock(idwhtoko, tglSTR, tglEND) # Mencari Stock awal
        df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].str.upper()
        df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
        df_transactions = df__transaction(idwhtoko, tglSTR, tglEND) # Mencari semua transaksi
        df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].str.upper()
        df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
        df_total = pd.concat([df_stock, df_transactions]).groupby('fv_artsizecode').sum().reset_index() # menjumlah stock awal dan transaksi
        df_total['fn_whconsiid'] = idwhtoko 
        df_total['tgltrx'] = tglEND
        
        df_tok = pd.concat([df_tok, df_total], ignore_index=True)
    fdate= str(year)[2:4]
    df_tok=df_tok.loc[~((df_tok['fn_whconsiid'].isin([1131,1130,1129]))&(df_tok['qty']==0))]
    df_tok= df_tok[['fn_whconsiid','fv_artsizecode','qty','tgltrx']].rename(columns={'qty':f'stock {bulan}_{fdate}'})
    
    if expot_excel == 'yes':
        max_rows = 1048570
        output_file = f'/opt/share/PB DB/pb_stock_{bulan}_{fdate}.xlsx'
        print(len(df_tok))
        print(max_rows)
        if len(df_tok) <= max_rows:
            # Simpan 1 sheet saja
            df_tok.to_excel(output_file, index=False)
        else:
            # Potong menjadi 2 bagian
            df1 = df_tok.iloc[:max_rows]
            df2 = df_tok.iloc[max_rows:]
            print(df1.info())
            print(df2.info())
            with pd.ExcelWriter(output_file) as writer:
                df1.to_excel(writer, sheet_name='Sheet1', index=False)
                df2.to_excel(writer, sheet_name='Sheet2', index=False)
        print(f'pb_stock_{bulan}_{fdate} Selesai')
        return df_tok
    print(f'pb_stock_{bulan}_{fdate} Selesai')
    return df_tok

def get_last_three_months_including_current():
    timestart = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("[INFO] Script get_last_three_months_including_current dimulai.")
    print(f"[INFO] Timestart: {timestart}\n")

    today = datetime.now()
    last_dates = []

    for i in range(2, -1, -1):  # Loop mundur dari 2 bulan lalu sampai bulan ini
        target_month = (today.month - i - 1) % 12 + 1
        target_year = today.year - ((today.month - i - 1) < 0)

        # Dapatkan hari terakhir bulan tersebut
        last_day = calendar.monthrange(target_year, target_month)[1]
        last_dates.append(f"{target_year}-{target_month:02d}-{last_day:02d}")
    for tgl in last_dates:
        get_stock_monthly(tgl, 'yes')
    # Print tanggal yang dipilih sebelum return
    print("[INFO] Tanggal yang dipilih:")
    print("\n".join(last_dates))

    return last_dates

In [ ]:
get_last_three_months_including_current()

In [21]:
tglEND='2025-08-31'
expot_excel='yes'


In [22]:
'''Main Stock Monthly'''
tglEND2= datetime.strptime(tglEND, "%Y-%m-%d").date()
tglEND = datetime.strptime(tglEND, '%Y-%m-%d').strftime('%Y-%m-%d 23:59:59')
# Dapatkan tanggal kemarin
today = datetime.now()
d2ago = today - timedelta(days=2)
# Ubah ke string (format default: YYYY-MM-DD)
# kemarin_str = kemarin.strftime("%Y-%m-%d")
# d2ago_str = d2ago.strftime("%Y-%m-%d")
year = int(tglEND[:4])
month = int(tglEND[5:7])
day = int(tglEND[8:10])
last_day = calendar.monthrange(year, month)[1]
if day == last_day:
    if month == 12:  
        month = 1
        year += 1
    else:
        month += 1
bulan = calendar.month_name[month].lower()[:3]
expot_excel = expot_excel.lower()
if expot_excel not in ['yes', 'no']:
    raise ValueError("Parameter 'expot_excel' harus berupa 'yes' atau 'no'.")
tglnow = datetime.today().date() # Tanggal Perhari Ini 

In [23]:

df_tokoawal = df__tokoawal()
idwhtoko_list_awal = df_tokoawal['fn_whconsiid'].tolist() # Ubah kedalam List

# Mencari tanggal SO terakhir utnuk masing-masing toko yang masih buka per tanggal yang dipilih
df_tglLSO = pd.concat([get_tgl(idwhtoko, tglnow, tglEND) for idwhtoko in idwhtoko_list_awal])
df_tglLSO['fd_tglflagIndo'] = pd.to_datetime(df_tglLSO['fd_tglflagIndo'])

if tglEND2 > tglnow:  # Jika tanggal yang dipilih lebih besar dari hari ini
    df_tglLSO.loc[df_tglLSO['fc_status']=="T",'fd_tglflagIndo'] -= pd.Timedelta(days=1)

In [24]:
df_tglLSO['fd_tglflagIndo'] = df_tglLSO['fd_tglflagIndo'].dt.strftime('%Y-%m-%d')
idwhtoko_list = df_tglLSO['fn_whconsiid'].tolist() # Ubah kedalam List untuk kode toko
tgl_list = df_tglLSO['fd_tglflagIndo'].tolist() # Ubah kedalam List untuk tanggal SO terakhir

In [25]:
def get_stock(idwhtoko, tglSTR, tglEND):
    df_stock = df__stock_so(idwhtoko, tglSTR, tglEND)
    if df_stock['qty'].sum() == 0:
        df_stock = df__stock_sa(idwhtoko, tglEND)
    return df_stock

In [26]:
def df__stock_so(idwhtoko, tglSTR, tglEND):
    query = '''
    select c.fv_artsizecode, IF(a.fn_qtyopname IS NULL, c.fn_qtyskrg, a.fn_qtyopname) AS qty
    from gis_db1.t_historysimpanstokopnamecounter a
    right join gis_db1.fl_stockarticleconsipostingmst b on a.fc_nodoc=b.fc_nodoc
    right join gis_db1.fl_stockarticleconsipostingdtl c on c.fc_nodoc=b.fc_nodoc
    where b.fn_whconsiid= '%s' AND SUBSTR(b.fd_tglflag, 1, 10) between %s AND %s
    GROUP BY c.fv_artsizecode
    '''
    with get_engine('read') as engine:  # Menggunakan context manager dengan with
                # Mencari tanggal Opname terakhir per tglnow
        df = pd.read_sql(query, engine, params=(idwhtoko, tglSTR, tglEND))
    return df

def df__stock_sa(idwhtoko, tglEND):
    query = '''
    select a.fv_artsizecode, a.fn_qty qty
    from saldoawalarticleconsi_tb a
    where a.fn_whconsiid= '%s' AND SUBSTR(fd_tglinject, 1, 10) <= %s
    '''
    with get_engine('read') as engine:  # Menggunakan context manager dengan with
                # Mencari tanggal Opname terakhir per tglnow
        df = pd.read_sql(query, engine, params=(idwhtoko, tglEND))
    return df

In [27]:
# Mencari Stock pertanggal yang dipilih dengan menjumlah stock awal SO terakhir dengan transaksi dari SO terakhir sampai tanggal yang dipilih
df_tok = pd.DataFrame() # Dataframe Kosong
for idwhtoko, tglSTR in zip(idwhtoko_list, tgl_list): # Looping untuk masing-masing toko dan tanggal SO terakhir
    df_stock = get_stock(idwhtoko, tglSTR, tglEND) # Mencari Stock awal
    df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].str.upper()
    df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
    df_transactions = df__transaction(idwhtoko, tglSTR, tglEND) # Mencari semua transaksi
    df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].str.upper()
    df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
    df_total = pd.concat([df_stock, df_transactions]).groupby('fv_artsizecode').sum().reset_index() # menjumlah stock awal dan transaksi
    df_total['fn_whconsiid'] = idwhtoko 
    df_total['tgltrx'] = tglEND
    
    df_tok = pd.concat([df_tok, df_total], ignore_index=True)
fdate= str(year)[2:4]
df_tok=df_tok.loc[~((df_tok['fn_whconsiid'].isin([1131,1130,1129]))&(df_tok['qty']==0))]
df_tok= df_tok[['fn_whconsiid','fv_artsizecode','qty','tgltrx']].rename(columns={'qty':f'stock {bulan}_{fdate}'})

In [17]:

if expot_excel == 'yes':
    max_rows = 1048570
    output_file = fr'\\192.168.1.254\Share\_From Valdi\PB DB\pb_stock_{bulan}_{fdate}.xlsx'
    print(len(df_tok))
    print(max_rows)
    if len(df_tok) <= max_rows:
        # Simpan 1 sheet saja
        df_tok.to_excel(output_file, index=False)
    else:
        # Potong menjadi 2 bagian
        df1 = df_tok.iloc[:max_rows]
        df2 = df_tok.iloc[max_rows:]
        print(df1.info())
        print(df2.info())
        with pd.ExcelWriter(output_file) as writer:
            df1.to_excel(writer, sheet_name='Sheet1', index=False)
            df2.to_excel(writer, sheet_name='Sheet2', index=False)
    print(f'pb_stock_{bulan}_{fdate} Selesai')
print(f'pb_stock_{bulan}_{fdate} Selesai')

939298
1048570
pb_stock_sep_25 Selesai
pb_stock_sep_25 Selesai


In [18]:
def df__customers_pb_n(placeholders):
        query = f'''
            SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
            where fc_code in ({placeholders})
            '''
        results = execute_raw_sql(query)
        df = pd.DataFrame(results)
        return df
df_cus=df__customers_pb_n(placeholders)
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
def change_cusid_to_whconsiid(df):
    df_cus=df__customers_pb_n(placeholders)
    df=pd.merge(df,df_cus,how='left',on='fn_cusid')
    df['fn_cusid']=df['fn_whconsiid']
    df=df.drop(columns='fn_whconsiid')
    df=df.rename(columns={'fn_cusid':'fn_whconsiid'})
    return df
def get_ostdo_(cus_list):
    query = f'''
        SELECT fv_artsizecode,fv_barcode from artikel_pb
        '''
    results = execute_raw_sql(query)
    df_art = pd.DataFrame(results)
    query_1= f'''SELECT 
    c.fn_cusid ,fv_artsizecode, sum(fn_qty) fn_qty
    FROM gis_db.dodtl_tb a 
    inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
    inner join gis_db.t_ordermst c on b.fc_noreff=fv_noorder
    left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
    where  fv_noreceiving is null and c.fn_cusid in ({cus_list}) and b.fc_status="F" 
    group by fn_cusid, fv_artsizecode, fd_date 
    '''
    # DatabaseConnector.execute_raw_sql(OutstandingDO_query, using=StockDashboardAPIView.dbsave)
    results_1 = DatabaseConnector.execute_raw_sql(query_1, using=StockDashboardAPIView.dbsave)
    df = pd.DataFrame(results_1)
    df=change_cusid_to_whconsiid(df)
    df=pd.merge(df, df_art, on='fv_artsizecode', how='left')
    df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()
    return df
OutstandingDO_df=get_ostdo_(cus_list)

NameError: name 'placeholders' is not defined

In [ ]:
# Main
import pandas as pd
import numpy as np

from datetime import date, timedelta
import calendar
from contextlib import contextmanager
from sqlalchemy import create_engine
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import os
from datetime import datetime
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb, df__artikel_pb_read, 
                             df__customers_pb_read, df__cus_user) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis,
                            get_artikel_md, get_new_or_changed_rows,upsert_data, process_data_cus)
# Proses
from modules.queries import (df__tglso, df__tglsa, df__disc2, df__margin, df__artsizecode, df__stock_so,
                             df__stock_sa, df__retur, df__size, generate_months_for_year, df__rtt)
from modules.db_connection import get_engine
from sqlalchemy import Integer, String, CHAR, Date, Text, text, Float
import numpy as np
import pandas as pd
from datetime import datetime
from sqlalchemy import MetaData
from sqlalchemy.dialects.mysql import insert as mysql_insert
from sqlalchemy.orm import sessionmaker
import traceback
import os

## (M) sales_to_sql

In [ ]:
year = 2023
start_time2= datetime.now().strftime('%Y-%m-%d %H:%M:%S')    
print("\n[INFO] Script sales_to_sql mulai dijalankan.")
print(f"[INFO] Timestamp: {start_time2}\n")

df1= df__penju(year)


In [ ]:
df= df1.copy()
df= clean_and_transform_data(df)
# df=insert_disc2_grouping(df,'sum')
# margin_tb(year)
# df=insert_margin(df,year)
# sum_col=['fn_jualpersize','total_penjualan2','total_disc2']
# agg_col=['fn_whconsiid', 'fv_artsizecode', 'tgltrx']
# fisrt_col=[col for col in df.columns if col not in sum_col + agg_col]
# col_col=fisrt_col+[col for col in df.columns if col not in fisrt_col]
# df=df[col_col]
# df['event']=df['event'].fillna(df['status_artikel'])
# df['fn_margin']=df['fn_margin'].fillna(0)
# df=rename_col_py_to_sql(df).sort_values(by='fn_idurut')
# df=insert_pk(df)
# df['fn_nett_amount'] = (df['fn_totalpenjualan']-df['fn_total_disc2']) * (1-df['fn_margin']/100)
# df['fn_sim_amount'] = np.where(df['fc_status'] == "X", df['fn_totalpenjualan'], df['fn_nett_amount'])
# df['fn_dpp'] = df['fn_nett_amount']/1.11
# df['fn_dpp2'] = df['fn_sim_amount']/1.11
# table_name = f"pb_penjualan_{year}"

### (P) insert_disc2_grouping


In [ ]:
df_disc2= df__disc2()
df = pd.merge(df, df_disc2, on='fv_nopenjualan', how='left')
df = df.loc[~df.duplicated()]
df['artsize_disc2nd'] = df['artsize_disc2nd'].fillna(0)
df['total_disc2']=df['artsize_disc2nd']*df['fn_jualpersize']
sum_col=['fn_jualpersize','total_penjualan2','total_disc2']
agg_col=['fv_nopenjualan','fn_whconsiid', 'fv_artsizecode', 'tgltrx']
fisrt_col=[col for col in df.columns if col not in sum_col + agg_col]
print(f" {sum_col}")
print(f" {fisrt_col}")

# df = df.groupby(['fv_nopenjualan','fn_whconsiid', 'fv_artsizecode', 'tgltrx']).agg({
#         **{col: 'sum' for col in sum_col} , **{col: 'first' for col in fisrt_col}
#         }).reset_index()

df = df.groupby(['fv_nopenjualan','fn_whconsiid', 'fv_artsizecode', 'tgltrx']).agg({
        **{col: 'sum' for col in sum_col} , **{col: 'first' for col in fisrt_col}
        }).reset_index()
# if type == 'sum':
#     df = df.groupby(['fv_nopenjualan','fn_whconsiid', 'fv_artsizecode', 'tgltrx']).agg({
#         **{col: 'sum' for col in sum_col} , **{col: 'first' for col in fisrt_col}
#         }).reset_index()
# else: df


In [ ]:
df.loc[df['fv_nopenjualan']=='PJ/DPS/2305/00014/77',
         ['fv_artsizecode','fn_hargasatuan','fn_jualpersize','total_penjualan2']]

In [ ]:
df1

## (M) upsert_customers_pb

In [ ]:
base_path = fr'\\192.168.1.254\Share\_From Valdi\PB DB\\'
excel_path = f'{base_path}customers_pb_sup.xlsx'
csv_path = f'{base_path}list user & employee.csv'
pic_path = f'{base_path}LIST PIC MARKETING.xlsx'
mic_path = f'{base_path}PIC MIC.xlsx'
table_name = 'customers_pb'
df = df__customers_pb_read()
df_excel = pd.read_excel(excel_path)
emp_tb=pd.read_csv(csv_path)
emp_tb['fn_userid']=emp_tb['fn_userid'].astype(int)
df_user=df__cus_user()
df_user=pd.merge(df_user,emp_tb[['fn_userid','fv_nameemployees']],on='fn_userid',how='left')
df_user['fn_cusid']=df_user['fn_cusid'].astype(int)
df=pd.merge(df,df_user[['fn_cusid','fv_nameemployees']],on='fn_cusid',how='left')
df=df.loc[~df['fn_whconsiid'].duplicated()]
df=pd.merge(df, df_excel, on='fn_cusid', how='left')
print(df.loc[df['fn_whconsiid']=="-",['fn_whconsiid']].value_counts())
df_picmkt=pd.read_excel(pic_path, header=3).dropna(how='all')
df_picmkt['SUPERVISOR/ OJT**'] = df_picmkt['SUPERVISOR/ OJT**'].str.replace(r'[^A-Za-z\s]', '', regex=True).str.strip().str.title()
df=pd.merge(df,df_picmkt[['MANAGER','SUPERVISOR/ OJT**','KODE TOKO GIS']],left_on='code',right_on='KODE TOKO GIS',how='left')
print(df.loc[df['fn_whconsiid']=="-",['fn_whconsiid']].value_counts())
df=df.rename(columns={'MANAGER_y':'MANAGER'})
df['SUPERVISOR']=df['SUPERVISOR/ OJT**']



In [ ]:
df_picmic=pd.read_excel(mic_path,).dropna(how='all')

In [ ]:
df=pd.merge(df,df_picmic[['PIC MIC','STORE GIS']],left_on='code',right_on='STORE GIS',how='left').fillna('-')
df['Alokator']=df['PIC MIC']
df.loc[df['fn_cusid']==915,'fn_whconsiid']= 9000

In [ ]:
df=df.drop(columns=['KODE TOKO GIS', 'MANAGER_x', 'SUPERVISOR/ OJT**','STORE GIS','PIC MIC'])

In [ ]:
df.info()

In [ ]:
print(df.loc[df['fn_cusid']==915,].value_counts())

In [ ]:
if df is None or df.empty:
        print("❌ DataFrame kosong! Upsert dibatalkan.")
else:
    print("✅ Data berhasil di-load dengan kolom:", df.columns.tolist())
df = process_data_cus(df)
df = df.where(pd.notnull(df), None)
upsert_data('imp',table_name,df, primary_key='customers_pb')

In [ ]:
df

## (M) stock and sales

In [5]:
from modules.queries import (df__penju, df__tokoawal, df__transaction, df__cus, get_monthly_sales_data, 
                             df__sales_thru_24, df__sales_thru_25, df__artsizecode, df__gdmd, fetch_data_margin,
                             df__sales_, df__customers_pb, df__artikel_pb, df__artikel_pb_read, 
                             df__customers_pb_read, df__cus_user, df__sales_invoice,  df__sales_bz_disc2nd,
                             df__sales_bz, df__sales_consi_disc2nd, df__sales_online_disc2nd, df__otsdm, df__otsdo,
                             df__artikel_status ) 
from modules.proses import (save_to_sql, add_index_and_foreign_keys, clean_and_transform_data, 
                            insert_disc2_grouping, insert_margin, rename_col_py_to_sql, insert_pk,
                            get_tgl, get_stock, get_stock_data, merge_and_calculate, merge_detail, 
                            generate_rename_dict,trx_rt, process_data_margin,rename_col_sql_to_xlsx_gis,
                            get_artikel_md, get_new_or_changed_rows,upsert_data, process_data_cus,
                            sls_disctok)
from main import md_artikel_size2
from datetime import date, timedelta

In [ ]:
def md_artikel_size2():
    df_artsq= df__artsizecode('imp')
    df_sqq24= df__sales_thru_24()
    df_sqq25= df__sales_thru_25()
    df_sg= df__gdmd()

    df_sq24=df_sqq24.copy()
    df_sq25=df_sqq25.copy()
    df_sq24=df_sq24.drop(columns=[df_sq24.columns[1]])
    df_sq25=df_sq25.drop(columns=[df_sq25.columns[1]])
    df_sq24['fn_artsizeid']=df_sq24['fn_artsizeid'].astype(str)
    df_sq25['fn_artsizeid']=df_sq25['fn_artsizeid'].astype(str)
    df_sq24 = df_sq24.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq24.columns[2:]].sum().reset_index()
    df_sq25 = df_sq25.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq25.columns[2:]].sum().reset_index()

    # Buat dictionary rename berdasarkan data
    df_sq24 = generate_rename_dict(2024, 2024, df_sq24)
    df_sq25 = generate_rename_dict(2025, 2025, df_sq25)
    df=pd.merge(df_sq24,df_sq25,how='outer',on=['fn_artsizeid','fv_artsizecode'])
    # df['qty retur']=df['qty retur_x'].fillna(0)+df['qty retur_y'].fillna(0)
    print(df.columns)
    df=df.rename(columns={
        'qty retur_x': 'retur 2024',
        'qty retur_y': 'retur 2025',})
    df['stock akhir']=df['stock akhir_y']
    df['total sales qty']=df['total sales qty_x'].fillna(0)+df['total sales qty_y'].fillna(0)
    df['total stock']=df['total sales qty']+df['stock akhir'].fillna(0)
    df=df.drop(columns=['total stock_x', 'stock akhir_x', 'stock akhir_y',
                        'total stock_y','total sales qty_x','total sales qty_y'])
    df_sg['fv_artsizecode']=df_sg['fv_artsizecode'].str.upper()
    df = df.groupby(['fv_artsizecode'], as_index=False).sum()
    df = pd.merge(df, df_sg, on='fv_artsizecode', how='outer')
    df = pd.merge(df, df_artsq, on='fv_artsizecode', how='left')
    df =generate_rename_dict(2025, 2025, df)
    df = df.loc[~(df['artikel'].isnull())]
    df2=df.fillna(0)
    df2=df2[df2['artikel']!=0]
    kolom_baru24 = ['artikel', 'size'] + [col for col in df2.columns if col not in ['artikel', 'size','fv_artsizecode','fn_artsizeid']]
    df2 = df2[kolom_baru24]
    df2r=df2.select_dtypes(include='number').sum()
    df2r.to_excel(fr'\\192.168.1.254\Share\_From Valdi\For MD\artsize_gd_result.xlsx')
    df2.to_excel(fr'\\192.168.1.254\Share\_From Valdi\For MD\stock artikel all 2025.xlsx', index=False)

In [ ]:
df_artsq= df__artsizecode('imp')
df_sqq24= df__sales_thru_24()
df_sqq25= df__sales_thru_25()
df_sg= df__gdmd()

df_sq24=df_sqq24.copy()
df_sq25=df_sqq25.copy()
df_sq24=df_sq24.drop(columns=[df_sq24.columns[1]])
df_sq25=df_sq25.drop(columns=[df_sq25.columns[1]])
df_sq24['fn_artsizeid']=df_sq24['fn_artsizeid'].astype(str)
df_sq25['fn_artsizeid']=df_sq25['fn_artsizeid'].astype(str)
df_sq24 = df_sq24.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq24.columns[2:]].sum().reset_index()
df_sq25 = df_sq25.groupby(['fv_artsizecode','fn_artsizeid'])[df_sq25.columns[2:]].sum().reset_index()

# Buat dictionary rename berdasarkan data
df_sq24 = generate_rename_dict(2024, 2024, df_sq24)
df_sq25 = generate_rename_dict(2025, 2025, df_sq25)
df=pd.merge(df_sq24,df_sq25,how='outer',on=['fn_artsizeid','fv_artsizecode'])
# df['qty retur']=df['qty retur_x'].fillna(0)+df['qty retur_y'].fillna(0)
print(df.columns)
df=df.rename(columns={
    'qty retur_x': 'retur 2024',
    'qty retur_y': 'retur 2025',})
df['stock akhir']=df['stock akhir_y']
df['total sales qty']=df['total sales qty_x'].fillna(0)+df['total sales qty_y'].fillna(0)
df['total stock']=df['total sales qty']+df['stock akhir'].fillna(0)
df=df.drop(columns=['total stock_x', 'stock akhir_x', 'stock akhir_y',
                    'total stock_y','total sales qty_x','total sales qty_y'])
df_sg['fv_artsizecode']=df_sg['fv_artsizecode'].str.upper()
df = df.groupby(['fv_artsizecode'], as_index=False).sum()
df = pd.merge(df, df_sg, on='fv_artsizecode', how='outer')
df = pd.merge(df, df_artsq, on='fv_artsizecode', how='left')
df =generate_rename_dict(2025, 2025, df)
df = df.loc[~(df['artikel'].isnull())]
df2=df.fillna(0)
df2=df2[df2['artikel']!=0]
kolom_baru24 = ['artikel', 'size'] + [col for col in df2.columns if col not in ['artikel', 'size','fv_artsizecode','fn_artsizeid']]
df2 = df2[kolom_baru24]
df2r=df2.select_dtypes(include='number').sum()
df2r.to_excel(fr'\\192.168.1.254\Share\_From Valdi\For MD\artsize_gd_result.xlsx')
df2.to_excel(fr'\\192.168.1.254\Share\_From Valdi\For MD\stock artikel all 2025.xlsx', index=False)

In [ ]:
md_artikel_size2()

In [ ]:
def get_stock_monthly(tglEND,expot_excel):
    '''Main Stock Monthly'''
    tglEND2= datetime.strptime(tglEND, "%Y-%m-%d").date()
    tglEND = datetime.strptime(tglEND, '%Y-%m-%d').strftime('%Y-%m-%d 23:59:59')
    # Dapatkan tanggal kemarin
    today = datetime.now()
    d2ago = today - timedelta(days=2)
    # Ubah ke string (format default: YYYY-MM-DD)
    # kemarin_str = kemarin.strftime("%Y-%m-%d")
    # d2ago_str = d2ago.strftime("%Y-%m-%d")
    year = int(tglEND[:4])
    month = int(tglEND[5:7])
    day = int(tglEND[8:10])
    last_day = calendar.monthrange(year, month)[1]
    if day == last_day:
        if month == 12:  
            month = 1
            year += 1
        else:
            month += 1
    bulan = calendar.month_name[month].lower()[:3]
    expot_excel = expot_excel.lower()
    if expot_excel not in ['yes', 'no']:
        raise ValueError("Parameter 'expot_excel' harus berupa 'yes' atau 'no'.")
    tglnow = datetime.today().date() # Tanggal Perhari Ini 
    df_tokoawal = df__tokoawal()
    idwhtoko_list_awal = df_tokoawal['fn_whconsiid'].tolist() # Ubah kedalam List

    # Mencari tanggal SO terakhir utnuk masing-masing toko yang masih buka per tanggal yang dipilih
    df_tglLSO = pd.concat([get_tgl(idwhtoko, tglnow, tglEND) for idwhtoko in idwhtoko_list_awal])
    df_tglLSO['fd_tglflagIndo'] = pd.to_datetime(df_tglLSO['fd_tglflagIndo'])

    if tglEND2 > tglnow:  # Jika tanggal yang dipilih lebih besar dari hari ini
        df_tglLSO.loc[df_tglLSO['fc_status']=="T",'fd_tglflagIndo'] -= pd.Timedelta(days=1)

    df_tglLSO['fd_tglflagIndo'] = df_tglLSO['fd_tglflagIndo'].dt.strftime('%Y-%m-%d')
    idwhtoko_list = df_tglLSO['fn_whconsiid'].tolist() # Ubah kedalam List untuk kode toko
    tgl_list = df_tglLSO['fd_tglflagIndo'].tolist() # Ubah kedalam List untuk tanggal SO terakhir

    # Mencari Stock pertanggal yang dipilih dengan menjumlah stock awal SO terakhir dengan transaksi dari SO terakhir sampai tanggal yang dipilih
    df_tok = pd.DataFrame() # Dataframe Kosong
    for idwhtoko, tglSTR in zip(idwhtoko_list, tgl_list): # Looping untuk masing-masing toko dan tanggal SO terakhir
        df_stock = get_stock(idwhtoko, tglSTR, tglEND) # Mencari Stock awal
        df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].str.upper()
        df_stock['fv_artsizecode'] = df_stock['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
        df_transactions = df__transaction(idwhtoko, tglSTR, tglEND) # Mencari semua transaksi
        df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].str.upper()
        df_transactions['fv_artsizecode'] = df_transactions['fv_artsizecode'].replace({'BBRD103V042333-': 'BBRD103V042333'})
        df_total = pd.concat([df_stock, df_transactions]).groupby('fv_artsizecode').sum().reset_index() # menjumlah stock awal dan transaksi
        df_total['fn_whconsiid'] = idwhtoko 
        df_total['tgltrx'] = tglEND
        
        df_tok = pd.concat([df_tok, df_total], ignore_index=True)
    fdate= str(year)[2:4]
    df_tok=df_tok.loc[~((df_tok['fn_whconsiid'].isin([1131,1130,1129]))&(df_tok['qty']==0))]
    df_tok= df_tok[['fn_whconsiid','fv_artsizecode','qty','tgltrx']].rename(columns={'qty':f'stock {bulan}_{fdate}'})
    
    if expot_excel == 'yes':
        max_rows = 1048570
        output_file = f'/opt/share/PB DB/pb_stock_{bulan}_{fdate}.xlsx'
        print(len(df_tok))
        print(max_rows)
        if len(df_tok) <= max_rows:
            # Simpan 1 sheet saja
            df_tok.to_excel(output_file, index=False)
        else:
            # Potong menjadi 2 bagian
            df1 = df_tok.iloc[:max_rows]
            df2 = df_tok.iloc[max_rows:]
            print(df1.info())
            print(df2.info())
            with pd.ExcelWriter(output_file) as writer:
                df1.to_excel(writer, sheet_name='Sheet1', index=False)
                df2.to_excel(writer, sheet_name='Sheet2', index=False)
        print(f'pb_stock_{bulan}_{fdate} Selesai')
        return df_tok
    print(f'pb_stock_{bulan}_{fdate} Selesai')
    return df_tok

In [ ]:
timestart = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("[INFO] Script get_last_three_months_including_current dimulai.")
print(f"[INFO] Timestart: {timestart}\n")

today = datetime.now()
last_dates = []

for i in range(2, -1, -1):  # Loop mundur dari 2 bulan lalu sampai bulan ini
    target_month = (today.month - i - 1) % 12 + 1
    target_year = today.year - ((today.month - i - 1) < 0)

    # Dapatkan hari terakhir bulan tersebut
    last_day = calendar.monthrange(target_year, target_month)[1]
    last_dates.append(f"{target_year}-{target_month:02d}-{last_day:02d}")
for tgl in last_dates:
    get_stock_monthly(tgl, 'yes')

In [ ]:
last_dates = get_last_three_months_including_current()

In [ ]:
last_dates

In [ ]:
def merge_detail(final_data, stock_data, start_year, end_year):
    df_articlesize = df__artsizecode('read')
    df_size = df__size()

    final_data = pd.merge(final_data, df_articlesize, on='fv_artsizecode', how='left')
    final_data = pd.merge(final_data, df_size, on='fn_sizeid', how='left')
    final_data = final_data[~final_data['fv_sizename'].isnull()]

    # Membuat kolom secara dinamis untuk semua bulan di rentang tahun
    columns = ['fn_artsizeid', 'fn_whconsiid', 'fv_artsizecode']
    rename_columns = {'qty': 'fn_qty_retur'}
    month_names = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

    current_year = datetime.now().year
    current_month = datetime.now().month +1
    # last_stock_col = None  # Variabel untuk menyimpan kolom stock terakhir

    for year in range(start_year, end_year + 1):
        last_month = current_month if year == current_year else 12  # Sampai bulan saat ini di tahun berjalan

        for month in range(1, last_month + 1):
            month_str = f"{month_names[month-1]}_{str(year)[-2:]}"  # Format jan_25, feb_25, dst.
            col_prefix = f'{month:02d}'
            
            stock_col = f'stock {month_str}'

            # Hanya menyimpan stock bulan terakhir
            if month == last_month:
                last_stock_col = stock_col
                columns.append(stock_col)
                rename_columns[stock_col] = 'fn_stock_at'
            else:
                total_stock_col = f'total_stock_{month_str}'
                qty_col = f'qty_{month_str}'
                
                columns.extend([stock_col, total_stock_col, qty_col])
                rename_columns.update({
                    qty_col: f'fn_qty_{col_prefix}',
                    stock_col: f'fn_stock_{col_prefix}',
                    total_stock_col: f'fn_ts_{col_prefix}'
                })
            
            if month == 12:
                columns[:-1]
                available_stock_cols = [col for col in stock_data.columns if col.startswith('stock ')]

                # Pilih kolom stok terakhir (sesuai urutan bulan)
                if available_stock_cols:
                    last_stock_col = (available_stock_cols)[-1]  # Ambil stok terakhir berdasarkan nama kolom
                else:
                    last_stock_col = None  # Jika tidak ada stok, gunakan None
                
                total_stock_dec = f'total_stock_dec_{str(year)[-2:]}'
                qty_dec = f'qty_dec_{str(year)[-2:]}'
                stock_dec=f'stock dec_{str(year)[-2:]}'
                columns.extend([total_stock_dec, qty_dec,last_stock_col])
                rename_columns.update({
                    stock_dec:'fn_stock_12',
                    total_stock_dec: 'fn_ts_12',
                    qty_dec: 'fn_qty_12',
                    last_stock_col:'fn_stock_at'
                })
    # additional condition
    if tgl_today=tgl1diawalbulan:
        columns.extend([])
        columns.extend([f'fn_stock_12','qty', 'total_stock', 'total_qty'])
        rename_columns.update({
            'total_qty': 'fn_total_sales_qty',
            'total_stock': 'fn_ts_all'})
    else:
        columns.extend([])
        columns.extend(['qty', 'total_stock', 'total_qty'])
        rename_columns.update({
            'total_qty': 'fn_total_sales_qty',
            'total_stock': 'fn_ts_all'
        })
    final_data = final_data[columns]
    final_data = final_data.rename(columns=rename_columns).sort_values(by='fn_whconsiid')
    return final_data

In [8]:
start_year=2025
end_year=2025
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("\n[INFO] Script stock_and_sales mulai dijalankan.")
print(f"[INFO] Timestamp: {timestamp}\n")
base_path = fr'\\192.168.1.254\Share\_From Valdi\PB DB\\'
print("Base path:", base_path)
print(os.path.exists(base_path)) 

stock_data_25 = get_stock_data(base_path, start_year, end_year)
print(stock_data_25.info())
print("stock_data prosess complite")

sales_data_25 = get_monthly_sales_data(start_year, end_year)
print(sales_data_25.columns)
print("Sales_data prosess complite")





[INFO] Script stock_and_sales mulai dijalankan.
[INFO] Timestamp: 2025-10-01 09:49:48

Base path: \\192.168.1.254\Share\_From Valdi\PB DB\\
True
Daftar file yang dicek: ['\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_jan_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_feb_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_mar_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_apr_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_may_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_jun_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_jul_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_aug_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_sep_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_oct_25.xlsx', '\\\\192.168.1.254\\Share\\_From Valdi\\PB DB\\\\pb_stock_nov_25.xlsx', '\\\\192.168.1.254\\Share\\_From Vald

c:\Users\IT-ANALIS\Documents\airflow-DPS\src\modules\proses.py:375: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  combined_data = combined_data.groupby(['fn_whconsiid', 'fv_artsizecode'], as_index=False).sum()


Setelah groupby: (1149058, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149058 entries, 0 to 1149057
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fn_whconsiid    1149058 non-null  int64  
 1   fv_artsizecode  1149058 non-null  object 
 2   stock jan_25    1149058 non-null  float64
 3   stock feb_25    1149058 non-null  float64
 4   stock mar_25    1149058 non-null  float64
 5   stock apr_25    1149058 non-null  float64
 6   stock may_25    1149058 non-null  float64
 7   stock jun_25    1149058 non-null  float64
 8   stock jul_25    1149058 non-null  float64
 9   stock aug_25    1149058 non-null  float64
 10  stock sep_25    1149058 non-null  float64
 11  stock oct_25    1149058 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 105.2+ MB
None
stock_data prosess complite
Index(['fn_whconsiid', 'fv_artsizecode', 'qty_jan_25', 'qty_feb_25',
       'qty_mar_25', 'qty_apr_2

c:\Users\IT-ANALIS\Documents\airflow-DPS\src\modules\queries.py:469: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  combined_sales = combined_sales.groupby(['fn_whconsiid', 'fv_artsizecode'], as_index=False).sum()


In [13]:
sales_data_25['qty_nov_25']=0

In [16]:
sales_data_25

,fn_whconsiid,fv_artsizecode,qty_jan_25,qty_feb_25,qty_mar_25,qty_apr_25,qty_may_25,qty_jun_25,qty_jul_25,qty_aug_25,qty_sep_25,qty_oct_25,qty_nov_25
0,1,AAAC096H0822L,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,AAAC096H0822M,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1,AAAC096H0822S,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,1,AAAC174M0323XXL,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1,AAAC287M0323L,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332216,1163,BABC352M1123M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
332217,1163,BBAC306M1123S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
332218,1163,GBRD138X112329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
332219,1163,GBTC203T122431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0


In [14]:
stock_data_25['stock nov_25']=stock_data_25['stock oct_25']

In [18]:
stock_data_25=stock_data_25.drop(columns='qty_nov_25')

In [19]:
# Menggabungkan dan menghitung
final_data_25 = merge_and_calculate(sales_data_25, stock_data_25, start_year, end_year)
print("final_data prosess complite")

final_data prosess complite


In [20]:


finale_25 = merge_detail(final_data_25, stock_data_25, start_year, end_year )
tanggal_selesai = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
print(f"finale prosess complite {tanggal_selesai}")

nama_tabel=f'sales_thru_pb_{str(start_year)[-2:]}'
# finale_25.to_sql(nama_tabel, engine2, if_exists='replace', index=False, dtype=sql_types)
save_to_sql(finale_25, nama_tabel, 'imp')
add_index_and_foreign_keys(
'imp',
nama_tabel,
index_cols=["fn_whconsiid", "fv_artsizecode"],
foreign_keys=[
    {
        "column": "fn_whconsiid",
        "ref_table": "customers_pb",
        "ref_column": "fn_whconsiid",
        "constraint_name": f"fk_whconsiid_pb_sales_thru_{start_year}"
    },
    {
        "column": "fv_artsizecode",
        "ref_table": "artikel_pb",
        "ref_column": "fv_artsizecode",
        "constraint_name": f"fk_artsize_pb_sales_thru_{start_year}"
    }
]
)
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("\n[INFO] Script telah di input di gis.local.")
print(f"[INFO] Timestamp: {timestamp}")
    # Mendapatkan tanggal saat ini

save_to_sql(finale_25, nama_tabel, 'read')
# add_index_and_foreign_keys(nama_tabel, ['fn_whconsiid', 'fv_artsizecode'])

tanggal_selesai = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    # Menambahkan pesan status berhasil
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("\n[INFO] Script telah berhasil dijalankan.")
print(f"[INFO] Timestamp: {timestamp}")

finale prosess complite 01-10-2025 10:00:44
✅ Index 'idx_fn_whconsiid_fv_artsizecode' berhasil ditambahkan.
✅ Foreign key 'fk_whconsiid_pb_sales_thru_2025' berhasil ditambahkan.
✅ Foreign key 'fk_artsize_pb_sales_thru_2025' berhasil ditambahkan.

[INFO] Script telah di input di gis.local.
[INFO] Timestamp: 2025-10-01 10:01:53

[INFO] Script telah berhasil dijalankan.
[INFO] Timestamp: 2025-10-01 10:04:57


In [ ]:
# finale_25.to_sql(nama_tabel, engine2, if_exists='replace', index=False, dtype=sql_types)
save_to_sql(finale_25, nama_tabel, 'imp')
add_index_and_foreign_keys(
'imp',
nama_tabel,
index_cols=["fn_whconsiid", "fv_artsizecode"],
foreign_keys=[
    {
        "column": "fn_whconsiid",
        "ref_table": "customers_pb",
        "ref_column": "fn_whconsiid",
        "constraint_name": f"fk_whconsiid_pb_{start_year}"
    },
    {
        "column": "fv_artsizecode",
        "ref_table": "artikel_pb",
        "ref_column": "fv_artsizecode",
        "constraint_name": f"fk_artsize_pb_{start_year}"
    }
]
)
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("\n[INFO] Script telah di input di gis.local.")
print(f"[INFO] Timestamp: {timestamp}")
    # Mendapatkan tanggal saat ini

save_to_sql(finale_25, nama_tabel, 'read')
# add_index_and_foreign_keys(nama_tabel, ['fn_whconsiid', 'fv_artsizecode'])

tanggal_selesai = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    # Menambahkan pesan status berhasil
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("\n[INFO] Script telah berhasil dijalankan.")
print(f"[INFO] Timestamp: {timestamp}")


# System API

In [ ]:
input_param={
            # 'alokator': alokator,
            'store_list': ['MD255'],
            # 'start': start.strftime("%Y-%m-%d"),
            # 'end': ends.strftime("%Y-%m-%d"),
            # 'type': tipe,
            # 'format': output_format,
            # 'artikel':artikel,
            # 'input_date': input_date.strftime("%Y-%m-%d"),
            # 'subid':subid,
            # 's_all':s_all,
            'divisi':'JEANS / YOUTH',
            'ornal':'Normal',
            # 'bulan':bulan,
            # 'category':category
        }

In [ ]:
def df__customers_pb_n(placeholders):
    query = f'''
        SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
        where fc_code in ({placeholders})
        '''
    with get_engine('imp') as engine:
        df= pd.read_sql(query, engine)
        return df

# 01 add whare artikel

In [ ]:
def add_where(input_param,cus='',art=''):
    if input_param['store_list'] is not None and input_param['divisi']!='':
        if store_list:
            placeholders = ", ".join([f"'{s}'" for s in store_list])
            df_cus = df__customers_pb_n(placeholders)
            whconsi_list = df_cus['fn_whconsiid'].tolist()
            
            if whconsi_list:
                whconsi_placeholders = ", ".join([str(s) for s in whconsi_list])
                whconsi = f"and {cus}fn_whconsiid IN ({whconsi_placeholders})"
            else: whconsi = ""  # atau kondisi default kalau kosong
    else: whconsi = ""  # jika input_param['store_list'] None

    if input_param['divisi'] is not None and input_param['divisi']!='':
        divisi = input_param['divisi']
        divisi = f'and {art}fv_divname="{divisi}"'
    else: divisi = ''

    if input_param['ornal'] is not None and input_param['divisi']!='':
        ornal = input_param['ornal']
        ornal = f'and {art}fv_ornal="{ornal}"'
    else: ornal = ''
    return {
        'whconsi':whconsi,
        'divisi':divisi,
        'ornal':ornal}

In [ ]:
where=add_where(input_param,cus='cus.')

In [ ]:
where

# Debug API

## Input dan def global

In [ ]:
from modules.db_connection import get_engine
input_value = datetime.today().strftime("%Y-%m-%d")
input_date = datetime.strptime(input_value, "%Y-%m-%d")
# Konversi kembali ke string
start=datetime.today()
ends=start - relativedelta(months=3)
# start_str = start.strftime("%Y-%m-%d")
# ends_str = ends.strftime("%Y-%m-%d")
alokator=''
store_list = ['MD255']
input_param={'alokator': alokator,
'store_list': store_list,
'start': start.strftime("%Y-%m-%d"),
'end': ends.strftime("%Y-%m-%d"),
'type': 'default',
# 'format': output_format,
'input_date': input_date.strftime("%Y-%m-%d"),
'artikel':'QY02066W'}

In [ ]:
whconsi_list=168

In [ ]:
def df__customers_pb_n(placeholders):
    query = f'''
        SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
        where fc_code in ({placeholders})
        '''
    with get_engine('imp') as engine:
        df= pd.read_sql(query, engine)
    # results = DatabaseConnector.execute_raw_sql(query, using=dbLoad)
    # df = pd.DataFrame(results)
    return df

In [ ]:
def df__all_customers(col):
    query = f'''
            SELECT {col} FROM gis_local.customers_pb
            '''
    with get_engine('imp') as engine:
        df= pd.read_sql(query, engine)
    return df

In [ ]:
def df_artikel_pb_(colom):
    query = f'''
        SELECT {colom}
        FROM gis_local.artikel_pb
        '''
    with get_engine('imp') as engine:
        df= pd.read_sql(query, engine)
    return df

## (Q) df__req_stock

In [ ]:
qstore_list=f'and fn_whconsiid in ({whconsi_list})'
query = f'''
        SELECT fn_whconsiid, (CAST(fn_qtykirim AS SIGNED) * -1) as stocks, fv_artsizecode from t_returdtl_pre t
        LEFT JOIN t_returmst_pre m on t.fv_noretur = m.fv_noretur
        WHERE t.fc_status in ('T') {qstore_list}
        '''

with get_engine('write') as engine:
        df_freestock= pd.read_sql(query, engine)

In [ ]:
df_freestock

## (Q) API retur filter artikel

In [ ]:
input_value = input_param['input_date']
alokator = input_param['alokator']
artikel=input_param['artikel']
query = f"""
select a.fn_whconsiid, fv_barcode, sum(qty)*-1 as retur FROM gis_local.trx_retur a
left join artikel_pb b on a.fv_artsizecode=b.fv_artsizecode 
left join customers_pb c on a.fn_whconsiid=c.fn_whconsiid
WHERE tgltrx <= '{input_value}' and fv_barcode in ('{artikel}')
group by fv_toko, fv_barcode
"""
with get_engine('imp') as engine:
        df_retur= pd.read_sql(query, engine)

In [ ]:
df_retur

## (Q) df__main

In [ ]:
input_value = input_param['input_date']
store_list = input_param['store_list']
if store_list:
    placeholders = ", ".join([f"'{s}'" for s in store_list])
df_cus = df__customers_pb_n(placeholders)
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
sales_thru = f"sales_thru_pb_{input_value[2:4]}"
trxstx = f"trxstx_pb{input_value[2:4]}"
stock = f"fn_stock_{input_value[5:7]}"

input_value = '2025-07-10'
whconsi_list=168

main_query_ = f"""
SELECT 
stock_backdate.fn_whconsiid,
SUM(stock) AS stocks,
stock_backdate.fv_artsizecode, 
fv_barcode, fv_sizename, fv_configname

FROM (
SELECT fn_whconsiid, {stock} AS stock, fv_artsizecode 
FROM gis_local.{sales_thru}
UNION ALL
SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
FROM gis_local.{trxstx} 
WHERE fd_tgltrx BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
UNION ALL
SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
FROM gis_local.trxrt_pb
WHERE fd_tgltrx BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
UNION ALL
SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode 
FROM gis_local.Stock_varian_25
WHERE fd_tgl BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
) stock_backdate
left join artikel_pb b on stock_backdate.fv_artsizecode = b.fv_artsizecode
WHERE 1=1 and fn_whconsiid in ({whconsi_list})
GROUP BY fn_whconsiid, fv_artsizecode
"""
with get_engine('imp') as engine:
    df_main= pd.read_sql(main_query_, engine)

In [ ]:
df_main

## (Q) df__trx_freestock

In [ ]:
input_value = '2025-07-11'
input_now = datetime.today().strftime("%Y-%m-%d")
# input_value = input_param['input_date']
trxstx = f"trxstx_pb{input_value[2:4]}"
# artikel=input_param['artikel']
artikel=None
if artikel==None:
    store_list = input_param['store_list']
    if store_list:
        placeholders = ", ".join([f"'{s}'" for s in store_list])
    df_cus = df__customers_pb_n(placeholders)
    cus_list= df_cus['fn_cusid'].tolist()
    cus_list =  ", ".join([f"'{s}'" for s in cus_list])
    # whconsi_list=df_cus['fn_whconsiid'].tolist()
    # whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
    qstore_list= f'and b.fn_whconsiid in ({whconsi_list})'
else:
    qstore_list= f'and fv_barcode in ("{artikel}")'
main_query = f"""
SELECT 
    stock_backdate.fn_whconsiid,
    SUM(stock) AS stocks,
    stock_backdate.fv_artsizecode
FROM (
    SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode , fv_stat stat
    FROM gis_local.{trxstx} 
    WHERE fd_tgltrx BETWEEN '{input_value}' AND '{input_now}'
    UNION ALL
    SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode , 'txrt' stat
    FROM gis_local.trxrt_pb 
    WHERE fd_tgltrx BETWEEN '{input_value}' AND '{input_now}'
    UNION ALL
    SELECT fn_whconsiid, fv_freestock AS stock, fv_artsizecode , 'var' stat
    FROM gis_local.Freestock_varian_25
    WHERE fd_tgl BETWEEN '{input_value}' AND '{input_now}'
) stock_backdate
LEFT JOIN artikel_pb a ON a.fv_artsizecode = stock_backdate.fv_artsizecode
LEFT JOIN customers_pb b ON stock_backdate.fn_whconsiid = b.fn_whconsiid
where 1=1 {qstore_list}
GROUP BY fn_whconsiid, fv_artsizecode
"""
with get_engine('imp') as engine:
    df_frees= pd.read_sql(main_query, engine)

In [ ]:
print(main_query_)

In [ ]:
print(main_query)

In [ ]:
df_frees.loc[(df_frees['fn_whconsiid']==168)]

In [ ]:
df_main.loc[df_main['fv_artsizecode']=='GB02378FS']

In [ ]:
df_combi=pd.concat([df_main, df_frees,df_freestock], ignore_index=True)

In [ ]:
df_result = df_combi.groupby(['fn_whconsiid', 'fv_artsizecode'], as_index=False)['stocks'].sum()


WB03395GV

In [ ]:
df_result

## (Q) df__main_filt_arts

In [ ]:
# input_value = input_param['input_date']
    # Konversi kembali ke string
input_value = '2025-07-10'
sales_thru = f"sales_thru_pb_{input_value[2:4]}"
trxstx = f"trxstx_pb{input_value[2:4]}"
stock = f"fn_stock_{input_value[5:7]}"
artikel=input_param['artikel']
whconsi_list=168
main_query = f"""
SELECT 
        stock_backdate.fn_whconsiid,
        SUM(stock) AS stocks,
        stock_backdate.fv_artsizecode,
        a.fv_barcode,
        a.fv_sizename,
        a.fv_configname,
        a.fv_namemark,
        b.fc_code,
        concat(fc_code, ' - ', fv_toko) fv_toko, stat
    FROM (
        SELECT fn_whconsiid, {stock} AS stock, fv_artsizecode, 'st' stat
        FROM gis_local.{sales_thru}
        UNION ALL
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, fv_stat stat
        FROM gis_local.{trxstx} 
        WHERE fd_tgltrx BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
        UNION ALL
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, 'trs' stat
        FROM gis_local.trxrt_pb 
        WHERE fd_tgltrx BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
        UNION ALL
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, 'var' stat
        FROM gis_local.Stock_varian_25
        WHERE fd_tgl BETWEEN '{input_value[0:4]}-{input_value[5:7]}-01' AND '{input_value}'
    ) stock_backdate
    LEFT JOIN artikel_pb a ON a.fv_artsizecode = stock_backdate.fv_artsizecode
    LEFT JOIN customers_pb b ON stock_backdate.fn_whconsiid = b.fn_whconsiid
    where fv_barcode in ('{artikel}')
    GROUP BY fn_whconsiid, fv_artsizecode
"""
with get_engine('imp') as engine:
    df= pd.read_sql(main_query, engine)

In [ ]:
input_value

In [ ]:
df

In [ ]:
df.loc[(df['fn_whconsiid']==168)]

In [ ]:
# if input_param==None:
qstore_list=''
# else:
#     store_list = input_param['store_list']
#     if store_list:
#         placeholders = ", ".join([f"'{s}'" for s in store_list])
#     df_cus = df__customers_pb_n(placeholders)
#     cus_list= df_cus['fn_cusid'].tolist()
#     cus_list =  ", ".join([f"'{s}'" for s in cus_list])
#     qstore_list= f'and fn_cusid in ({cus_list})'
query = f'''
        SELECT fv_artsizecode,fv_barcode from artikel_pb
        '''
with get_engine('imp') as engine:
    df= pd.read_sql(query, engine)
# results = DatabaseConnector.execute_raw_sql(query)
# df_art = pd.DataFrame(results)
query_1= f'''SELECT 
    c.fn_cusid ,fv_artsizecode, sum(fn_qty) OstDO
    FROM gis_db.dodtl_tb a 
    inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
    inner join gis_db.t_ordermst c on b.fc_noreff=fv_noorder
    left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
    where  fv_noreceiving is null {qstore_list} and b.fc_status="F" 
    group by fn_cusid, fv_artsizecode, fd_date 
'''
with get_engine('write') as engine:
    df= pd.read_sql(query_1, engine)

In [ ]:
print(main_query)

In [ ]:
input_now = datetime.today().strftime("%Y-%m-%d")
input_value = '2025-05-05'

In [ ]:
main_query = f"""
SELECT 
        stock_backdate.fn_whconsiid,
        SUM(stock) AS stocks,
        stock_backdate.fv_artsizecode,
        a.fv_barcode,
        a.fv_sizename,
        a.fv_configname,
        a.fv_namemark,
        b.fc_code,
        concat(fc_code, ' - ', fv_toko) fv_toko, stat
    FROM (
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, fv_stat stat
        FROM gis_local.{trxstx} 
        WHERE fd_tgltrx BETWEEN '{input_value}' AND '{input_now}'
        UNION ALL
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, 'trs' stat
        FROM gis_local.trxrt_pb 
        WHERE fd_tgltrx BETWEEN '{input_value}' AND '{input_now}'
        UNION ALL
        SELECT fn_whconsiid, fn_stock AS stock, fv_artsizecode, 'var' stat
        FROM gis_local.Stock_varian_25
        WHERE fd_tgl BETWEEN '{input_value}' AND '{input_now}'
    ) stock_backdate
    LEFT JOIN artikel_pb a ON a.fv_artsizecode = stock_backdate.fv_artsizecode
    LEFT JOIN customers_pb b ON stock_backdate.fn_whconsiid = b.fn_whconsiid
    where fv_barcode in ('{artikel}')
    GROUP BY fn_whconsiid, fv_artsizecode, stat
"""
with get_engine('imp') as engine:
    df_trx= pd.read_sql(main_query, engine)

In [ ]:
query = '''SELECT * FROM gis_local.Stock_varian_25;
'''
with get_engine('imp') as engine:  # Menggunakan context manager dengan with
                # Mencari tanggal Opname terakhir per tglnow
        dvar = pd.read_sql(query, engine)

## (Q) API df_sales

In [ ]:
start_str = input_param['start']
ends_str = input_param['end']
artikel=input_param['artikel']
query = f"""
    select fv_barcode, sum(fn_jualpersize) as sales, fn_jualpersize, fv_sizename, fn_whconsiid, trx.fv_artsizecode from (
    SELECT fv_barcode, fn_jualpersize, fd_tgltrx, fv_sizename, fn_whconsiid, a.fv_artsizecode FROM gis_local.pb_penjualan_2025 a
    left join artikel_pb b on a.fv_artsizecode=b.fv_artsizecode where fv_barcode in ('{artikel}')
    union all 
    SELECT fv_barcode, fn_jualpersize, fd_tgltrx, fv_sizename, fn_whconsiid, a.fv_artsizecode FROM gis_local.pb_penjualan_2024 a
    left join artikel_pb b on a.fv_artsizecode=b.fv_artsizecode where fv_barcode in ('{artikel}')) trx
    where trx.fd_tgltrx between '{ends_str}' and '{start_str}' 
    group by fv_barcode, fn_whconsiid"""
with get_engine('imp') as engine:
    df_sales= pd.read_sql(query, engine)

In [ ]:
df_sales

## (P) API proc_sales_stock

In [ ]:
input_value = input_param['input_date']
artikel=input_param['artikel']
query = f"""
SELECT a.fn_whconsiid, fv_barcode, fn_stock, fd_tgltrx FROM 
(SELECT * FROM gis_local.trxstx_pb24 
where fv_stat like 'receive%%' and fd_tgltrx <= '{input_value}'
union all
SELECT * FROM gis_local.trxstx_pb25 
where fv_stat like 'receive%%' and fd_tgltrx <= '{input_value}') a
left join artikel_pb b on a.fv_artsizecode = b.fv_artsizecode
WHERE 1=1 and fv_barcode in ('{artikel}')
group by fn_whconsiid,fv_barcode, fd_tgltrx 
"""
with get_engine('imp') as engine:
    df_aging= pd.read_sql(query, engine)

In [ ]:
if not df_aging.empty:
    # Group df_aging by fn_whconsiid and fv_barcode, and get the minimum fd_tgltrx
    aging_min_date = df_aging.groupby(['fn_whconsiid', 'fv_barcode'])['fd_tgltrx'].min().reset_index()
    aging_min_date.rename(columns={'fd_tgltrx': '1st DO'}, inplace=True)

    # Group df_aging by fn_whconsiid and fv_barcode, and get the maximum fd_tgltrx (last_receive)
    aging_max_date = df_aging.groupby(['fn_whconsiid', 'fv_barcode'])['fd_tgltrx'].max().reset_index()
    aging_max_date.rename(columns={'fd_tgltrx': 'last_receive'}, inplace=True)

    # Merge df with aging_min_date on fn_whconsiid and fv_barcode
    df = pd.merge(df, aging_min_date, on=['fn_whconsiid', 'fv_barcode'], how='left')
    df = pd.merge(df, aging_max_date, on=['fn_whconsiid', 'fv_barcode'], how='left')

    # Get stock on 1st DO
    # Merge aging_df with aging_min_date to get stock on 1st DO
    aging_stock_on_1st_do = pd.merge(
        df_aging,
        aging_min_date,
        on=['fn_whconsiid', 'fv_barcode'],
        how='left'
    )

    # Filter rows where fd_tgltrx matches the 1st DO
    aging_stock_on_1st_do = aging_stock_on_1st_do[
    aging_stock_on_1st_do['fd_tgltrx'] == aging_stock_on_1st_do['1st DO']]

    # Select only the required columns
    aging_stock_on_1st_do = aging_stock_on_1st_do[['fn_whconsiid', 'fv_barcode', 'fn_stock']]
    # aging_stock_on_1st_do.rename(columns={'fn_stock': 'stock_on_1st_do'}, inplace=True)

    # Merge df with aging_stock_on_1st_do to add stock_on_1st_do column
    df = pd.merge(df, aging_stock_on_1st_do, on=['fn_whconsiid', 'fv_barcode'], how='left')
    df_sum = df.groupby(['fn_whconsiid', 'fv_barcode'], as_index=False)['fn_stock'].sum().drop_duplicates(subset=['fv_barcode'])
    df = pd.merge(df, df_sum, on=['fn_whconsiid', 'fv_barcode'], how='left')
    df = df.rename(columns={'fn_stock_y': 'stock_on_1st_do'}).drop(columns=['fn_stock_x'])
    df['stock_on_1st_do'] = df['stock_on_1st_do'].fillna(0)
    # Convert '1st DO' to datetime
    df['1st DO'] = pd.to_datetime(df['1st DO'])
    df['last_receive'] = pd.to_datetime(df['last_receive'])

    # Hitung aging (selisih hari dari last_receive ke hari ini)
    df['aging'] = (pd.Timestamp.today().normalize() - df['last_receive']).dt.days.fillna(0).astype(int)

    # Format '1st DO' to 'YYYY-MM-DD'
    df['1st DO'] = df['1st DO'].dt.strftime('%Y-%m-%d')
    df['last_receive'] = df['last_receive'].dt.strftime('%Y-%m-%d')
else:
    # Jika aging_df kosong, buat kolom dengan nilai default
    df['1st DO'] = '2024-01-01'
    df['last_receive'] = '2024-01-01'
    df['aging'] = '2024-01'
    df['stock_on_1st_do'] = 0

In [ ]:
all_sizes_bottom = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
                        '39', '40', '42', '44', 'ALLSIZE']
all_sizes_top = ['FS', 'XS', 'SS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
all_sizes = all_sizes_bottom + all_sizes_top

# Buat pivot table
pivot_table = pd.pivot_table(
    df,
    values='stocks',
    index=['fv_barcode', 'fn_whconsiid'],
    columns=['fv_sizename'],
    aggfunc='sum',
    fill_value=0
)

# Reset index agar bisa merge
pivot_table = pivot_table.reset_index()

# Kolom detail yang ingin ditambahkan kembali
kolom_detail = ['fv_barcode', '1st DO', 'last_receive', 'aging','fv_configname',
                'stock_on_1st_do', 'fn_whconsiid']
# Merge detail
pivot_table = pd.merge(pivot_table, df[kolom_detail], on=['fv_barcode', 'fn_whconsiid'], how='left')
pivot_table = pivot_table.drop_duplicates()

# Urutkan kolom agar kolom ukuran di akhir
urutan_kolom = kolom_detail + [col for col in pivot_table.columns if col not in kolom_detail]
pivot_table = pivot_table[urutan_kolom]

# Pastikan semua ukuran muncul
for size in all_sizes:
    if size not in pivot_table.columns:
        pivot_table[size] = 0
    pivot_table[size] = pd.to_numeric(pivot_table[size], errors='coerce').fillna(0).astype(int)

# Atur ulang urutan: non-size columns dulu, lalu size columns
size_columns = [size for size in all_sizes if size in pivot_table.columns]
other_columns = [col for col in pivot_table.columns if col not in size_columns]
pivot_table = pivot_table[other_columns + size_columns]

# Hitung status (BROKEN / SEHAT)
top_columns = ["L", "M", "S", "XL", "XS", "XXL", "XXXL"]
bottom_columns = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
                    '35', '36', '37', '38', '39', '40', '42', '44']

pivot_table['status'] = np.where(
    (pivot_table["fv_configname"] == "TOP") & (pivot_table[top_columns].gt(0).sum(axis=1) < 3), "BROKEN",
    np.where(
        (pivot_table["fv_configname"] == "BOTTOM") & (pivot_table[bottom_columns].gt(0).sum(axis=1) < 5), "BROKEN",
        "SEHAT"
    )
)

# Hitung total jumlah qty size (sumQty)
pivot_table['sumQty'] = pivot_table[size_columns].sum(axis=1)
pivot_table=pivot_table.drop(columns=['fv_configname'])

In [ ]:
pivot_table

In [ ]:
df=pivot_table

## (P) API proc_sales_stock

In [ ]:
if not df_sales.empty:
    # Merge df with df_sales on 'fv_barcode' and 'fn_whconsiid'
    df = pd.merge(df, df_sales[['fv_barcode', 'sales', 'fn_whconsiid']], on=['fv_barcode', 'fn_whconsiid'], how='left')
    df['sales'] = df['sales'].fillna(0)
    # Hitung total stock per barcode
    df['sumQty'] = pd.to_numeric(df['sumQty'], errors='coerce')
    df['sales'] = pd.to_numeric(df['sales'], errors='coerce')
    df['sumQty'] = df['sumQty'].fillna(0)
    df['sales'] = df['sales'].fillna(0)
    df['stockAwal'] = df['sumQty'] + df['sales']
    df = df.reset_index(drop=True)
else: 
    df['sales'] = 0
    df['sumQty'] = 0
    df['stockAwal'] = df['sumQty'] + df['sales']

In [ ]:
df

In [ ]:
if not df_retur.empty:
    # Merge df with df_retur on 'fv_barcode' and 'fn_whconsiid'
    df = pd.merge(df, df_retur[['fv_barcode', 'retur', 'fn_whconsiid']], on=['fv_barcode', 'fn_whconsiid'], how='left')
    df['retur'] = df['retur'].fillna(0)
else:
    df['retur'] = 0

In [ ]:
df

In [ ]:
df.loc[(df['fv_artsizecode']=='AAAC096H0822S') & (df['fn_whconsiid']==1)]

## (Q) df_main_draft

In [ ]:
whconsi_list=73

In [ ]:
input_value = input_param['input_date']
store_list = input_param['store_list']
if store_list:
    placeholders = ", ".join([f"'{s}'" for s in store_list])
df_cus = df__customers_pb_n(placeholders)
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
sales_thru = f"sales_thru_pb_{input_value[2:4]}"
trxstx = f"trxstx_pb{input_value[2:4]}"
stock = f"fn_stock_{input_value[5:7]}"

main_query = f"""
SELECT 
fn_whcosiid_asal fn_whconsiid,
SUM(fn_qtykirim) AS stocks,
a.fv_artsizecode, fv_barcode, fv_sizename
from gis_local.t_draftdtl a
left join artikel_pb b on a.fv_artsizecode = b.fv_artsizecode
WHERE 1=1 and fn_whcosiid_asal in ({whconsi_list})
GROUP BY fn_whcosiid_asal , a.fv_artsizecode
"""
with get_engine('imp') as engine:
    df= pd.read_sql(main_query, engine)

In [ ]:
kolom_formalitas = ['1st DO', 'last_receive', 'aging', 'fv_configname', 'stock_on_1st_do']

# Tambahkan kolom kosong jika belum ada
for kolom in kolom_formalitas:
    if kolom not in df.columns:
        df[kolom] = None  # atau np.nan jika ingin float kosong

In [ ]:
all_sizes_bottom = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
                    '39', '40', '42', '44', 'ALLSIZE']
all_sizes_top = ['FS', 'XS', 'SS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
all_sizes = all_sizes_bottom + all_sizes_top

# Buat pivot table
pivot_table = pd.pivot_table(
    df,
    values='stocks',
    index=['fv_barcode', 'fn_whconsiid'],
    columns=['fv_sizename'],
    aggfunc='sum',
    fill_value=0
)

# Reset index agar bisa merge
pivot_table = pivot_table.reset_index()

# Kolom detail yang ingin ditambahkan kembali
kolom_detail = ['fv_barcode', '1st DO', 'last_receive', 'aging','fv_configname',
                'stock_on_1st_do', 'fn_whconsiid']
# Merge detail
pivot_table = pd.merge(pivot_table, df[kolom_detail], on=['fv_barcode', 'fn_whconsiid'], how='left')
pivot_table = pivot_table.drop_duplicates()

# Urutkan kolom agar kolom ukuran di akhir
urutan_kolom = kolom_detail + [col for col in pivot_table.columns if col not in kolom_detail]
pivot_table = pivot_table[urutan_kolom]

# Pastikan semua ukuran muncul
for size in all_sizes:
    if size not in pivot_table.columns:
        pivot_table[size] = 0
    pivot_table[size] = pd.to_numeric(pivot_table[size], errors='coerce').fillna(0).astype(int)

# Atur ulang urutan: non-size columns dulu, lalu size columns
size_columns = [size for size in all_sizes if size in pivot_table.columns]
other_columns = [col for col in pivot_table.columns if col not in size_columns]
pivot_table = pivot_table[other_columns + size_columns]

# Hitung status (BROKEN / SEHAT)
top_columns = ["L", "M", "S", "XL", "XS", "XXL", "XXXL"]
bottom_columns = ['25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
                    '35', '36', '37', '38', '39', '40', '42', '44']

pivot_table['status'] = np.where(
    (pivot_table["fv_configname"] == "TOP") & (pivot_table[top_columns].gt(0).sum(axis=1) < 3), "BROKEN",
    np.where(
        (pivot_table["fv_configname"] == "BOTTOM") & (pivot_table[bottom_columns].gt(0).sum(axis=1) < 5), "BROKEN",
        "SEHAT"
    )
)

# Hitung total jumlah qty size (sumQty)
pivot_table['sumQty'] = pivot_table[size_columns].sum(axis=1)
pivot_table=pivot_table.drop(columns=['fv_configname'])

In [ ]:
pivot_table

In [ ]:
df=pivot_table
type_table='detail'
col='fv_barcode'
df_cus = df__all_customers("concat(fc_code, ' - ', fv_toko) fv_toko, fn_whconsiid, fc_code")
if col == 'fv_artsizecode':
    df_art= df_artikel_pb_('fv_barcode,fv_catname,fv_brandname,fv_divname,fv_colorname,fv_rating,fv_configname,fv_namemark,fm_price,fv_sizename,fv_artsizecode,fv_fitname')
    df = pd.merge(df, df_art, on='fv_artsizecode', how='left')
else:
    df_art= df_artikel_pb_('fv_barcode,fv_catname,fv_brandname,fv_divname,fv_colorname,fv_rating,fv_configname,fv_namemark,fm_price,fv_status').drop_duplicates('fv_barcode')
    df = pd.merge(df, df_art, on='fv_barcode', how='left')
df = pd.merge(df, df_cus, on='fn_whconsiid', how='left')
if type_table == 'mobile_view':
    df['fv_sizename_x']=df['fv_sizename_x'].fillna(df['fv_sizename_y'])
    df=df.rename(columns={'fv_sizename_x': 'fv_sizename'})
df['artikel']= df['fv_barcode']
df['fv_artsizecode']= df.empty

In [ ]:
df=df[['fv_toko',
        'fv_barcode',
        '25',
        '26',
        '27',
        '28',
        '29',
        '30',
        '31',
        '32',
        '33',
        '34',
        '35',
        '36',
        '37',
        '38',
        '39',
        '40',
        '42',
        '44',
        'ALLSIZE',
        'FS',
        'XS',
        'SS',
        'S',
        'M',
        'L',
        'XL',
        'XXL',
        'XXXL',
        'fn_whconsiid',
        ]]

In [ ]:
df

In [ ]:
print(main_query)

In [ ]:
pic_mic_dict = {
    3079: 'IREN',
    26: 'TYAS / YUNI',
    3348: 'TYAS / YUNI',
    200: 'VINDRA',
    1585: 'DIAN/ARUM',
    3107: 'NOPAL',
    147: 'SHOFI',
    3350: 'IMAS / ANGGI',
    1804: 'MILI/NENI',
    3341: 'MILI/NENI',
    1759: 'NOVA',
    40: 'SYAFII',
    1385: 'RILYA'
}
# Buat dict baru dengan key = user id, value = nama PIC
# user_id_to_name = {uid: name for name, uid in pic_mic_dict.items() if uid is not None}


In [ ]:
pic_mic_dict

In [ ]:
user_id = pic_mic_dict[3079]
print(user_id)


In [ ]:
df_main

# AR

In [ ]:
query = f'''SELECT distinct main.fv_nomutasi, fd_tgltransaksi, a.fc_nodoc, c.fv_noreceiving, fd_tglreceive, fn_whconsiid, fn_whconsiid2, d.fv_artsizecode, sub_main.fn_qtykirim, fn_qtyterima FROM gis_db.t_mutasimst main

left join gis_db.domst_tb a on main.fv_nomutasi=  a.fc_noreff 
left join gis_db.t_mutasidtl sub_main on main.fv_nomutasi=  sub_main.fv_nomutasi
left join gis_db.t_receivingmst c on a.fc_nodoc=  c.fc_nosj
left join gis_db.t_receivingdtl d on c.fv_noreceiving=  d.fv_noreceiving and sub_main.fv_artsizecode=d.fv_artsizecode
where a.fd_date between '2025-04-01 00:00:00' and '2025-09-31 23:59:59' and 
a.fc_status <> "F"'''
with get_engine('write') as engine:  # Menggunakan context manager dengan with
    df = pd.read_sql(query, engine)

In [ ]:
df_cuspb=df__customers_pb('fn_whconsiid,fv_toko "Toko Pengirim"')

In [ ]:
df_cuspb

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df=pd.merge(df,df_cuspb, on='fn_whconsiid', how='left')

In [ ]:
df_cuspb=df__customers_pb('fn_whconsiid,fv_toko "Toko Terima"')

In [ ]:
df=pd.merge(df,df_cuspb, left_on='fn_whconsiid2', right_on='fn_whconsiid', how='left')

In [ ]:
name_col = {
        'fv_nomutasi': 'No Mutasi',
        'fc_nodoc': 'No DM',
        'fv_noreceiving': 'No Receiving',
        'fv_artsizecode': 'Artikel Size',
        'fn_qtykirim':'Qty Kirim',
        'fn_qtyterima': 'Qty Terima',
        'fd_tgltransaksi':'Tgl Mutasi',
        'fd_tglreceive':'Tgl Receive'}
    # Lakukan rename dan sort
df = df.rename(columns=name_col)


In [ ]:
df=df[['No Mutasi','Tgl Mutasi','No DM','No Receiving','Tgl Receive','Toko Pengirim','Toko Terima','Artikel Size','Qty Kirim','Qty Terima'
    
]]

In [ ]:
df.to_excel('riwayat mutasi.xlsx',index=False)

# ML

In [ ]:
query = f'''select a.fv_artsizecode, a.fn_qtyopname qty, b.fc_status, fn_qtyskrg qty_fl
    from gis_db1.t_historysimpanstokopnamecounter a
    right join gis_db1.fl_stockarticleconsipostingmst b on a.fc_nodoc=b.fc_nodoc
    right join gis_db1.fl_stockarticleconsipostingdtl c on c.fc_nodoc=b.fc_nodoc
    where b.fn_whconsiid= 84 AND SUBSTR(b.fd_tglflag, 1, 10) between '2025-07-01 00:00:01' AND '2025-09-31 00:00:01'
    GROUP BY a.fv_artsizecode'''
with get_engine('read') as engine:  # Menggunakan context manager dengan with
    df = pd.read_sql(query, engine)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# ==== Contoh data sederhana ====
data = {
    "Year": [2023]*12 + [2024]*12,
    "Month": list(range(1,13))*2,
    "Category": ["Shirt"]*24,
    "Size": ["M"]*24,
    "Price": [200,200,200,180,200,200,200,190,190,200,200,200] * 2,
    "Season": ["Rainy","Rainy","Dry","Dry","Dry","Dry","Dry","Dry","Dry","Rainy","Rainy","Rainy"]*2,
    "Promo": [0,0,1,0,0,0,1,0,0,0,1,0]*2,
    "Holiday": [0,0,0,1,0,0,0,0,0,0,0,0]*2,
    "Qty_Sold": [
        # 2023 (training)
        120,130,180,220,160,150,200,210,205,190,195,210,
        # 2024 (actual for evaluation)
        125,135,190,230,170,160,210,220,215,200,210,225
    ]
}
df = pd.DataFrame(data)

# ==== Pilih kategori & size tertentu ====
cat = "Shirt"
size = "M"
df_cs = df[(df["Category"]==cat) & (df["Size"]==size)].copy()

# Encode categorical (Season)
df_cs = pd.get_dummies(df_cs, columns=["Season"], drop_first=True)

# Feature set
features = ["Month","Price","Promo","Holiday","Season_Rainy"]
X_train = df_cs[df_cs["Year"]==2023][features]
y_train = df_cs[df_cs["Year"]==2023]["Qty_Sold"]

X_test = df_cs[df_cs["Year"]==2024][features]
y_test = df_cs[df_cs["Year"]==2024]["Qty_Sold"]

# ==== Model ====
model = XGBRegressor(objective="reg:squarederror", n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# ==== Evaluasi ====
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE (Mean Absolute Error): {mae:.2f}")

# ==== Plot ====
plt.figure(figsize=(8,5))
plt.plot(X_train["Month"], y_train, label="Actual 2023", marker="o")
# plt.plot(X_test["Month"], y_test, label="Actual 2024", marker="o")
plt.plot(X_test["Month"], y_pred, label="Predicted 2024", marker="x")
plt.xlabel("Month")
plt.ylabel("Qty Sold")
plt.title(f"Forecast vs Actual - {cat} Size {size}")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# ==== contoh data ====
data = {
    "Month": [1,2,3,4,5],
    "Category": ["Shirt","Shirt","Shirt","Pants","Pants"],
    "Size": ["M","M","L","M","L"],
    "Color": ["Black","White","Blue","Black","White"],
    "Fitting": ["Regular","Slim","Oversize","Regular","Slim"],
    "Price": [199000,229000,199000,259000,279000],
    "Season": ["Rainy","Rainy","Holiday","Dry","Dry"],
    "Promo": [0,1,1,0,1],
    "Holiday": [0,0,1,0,0],
    "Qty_Sold": [120,170,210,90,130]
}
df = pd.DataFrame(data)

# ==== definisi fitur & target ====
X = df.drop("Qty_Sold", axis=1)
y = df["Qty_Sold"]

# Kolom kategorikal
categorical = ["Category","Size","Color","Fitting","Season"]
numeric = ["Month","Price","Promo","Holiday"]

# One-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), categorical),
        ("num", "passthrough", numeric)
    ]
)

# ==== Model 1: Linear Regression untuk interpretasi ====
linreg_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

linreg_model.fit(X, y)

# Ambil koefisien Linear Regression
feature_names = (linreg_model.named_steps["preprocessor"]
                 .get_feature_names_out())
coef = linreg_model.named_steps["regressor"].coef_

coef_df = pd.DataFrame({"Feature": feature_names, "Coefficient": coef})
print("=== Pengaruh Fitur (Linear Regression) ===")
print(coef_df.sort_values(by="Coefficient", ascending=False))

# ==== Model 2: XGBoost untuk prediksi ====
xgb_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(n_estimators=50, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

print("\n=== Hasil Prediksi vs Data Asli (XGBoost) ===")
comparison = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
print(comparison)

# Plot
comparison.plot(kind="bar", figsize=(6,4))
plt.title("Prediksi vs Aktual")
plt.ylabel("Qty Sold")
plt.show()


# Last Prog